In [1]:
from logger import MyLogger
import SubjObjLoader
import json
from torch import nn
import torch
import importlib
from collections import deque
# import dataloader
from BertRDMLoader import *
import json
from torch import nn
import torch
from transformers import BertModel, BertConfig, BertTokenizer
import importlib
from tensorboardX import SummaryWriter
import torch.nn.utils.rnn as rnn_utils
import tsentiLoader

In [2]:
import emotionLoader
from SubjObjLoader import *
import numpy as np

In [3]:
from sklearn.metrics import accuracy_score

### BERT RDM CM 模型代码
> 需要改动的地方
> - 损失函数要从训练函数中拆分出来，方便后面的联合训练
> 

In [14]:
class pooling_layer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(pooling_layer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def forward(self, inputs):
        assert(inputs.ndim == 4 ) # [batchsize, max_seq_len, max_word_num, input_dim] 
        batch_size, max_seq_len, max_word_num, input_dim = inputs.shape
        assert(input_dim == self.input_dim)
        t_inputs = inputs.reshape([-1, self.input_dim])
        return self.linear(t_inputs).reshape(
            
            [-1, max_word_num, self.output_dim]
        
        ).max(axis=1)[0].reshape(
        
            [-1, max_seq_len, self.output_dim]
        
        )

class RDM_Model(nn.Module):
    def __init__(self, word_embedding_dim, sent_embedding_dim, hidden_dim, dropout_prob):
        super(RDM_Model, self).__init__()
        self.embedding_dim = sent_embedding_dim
        self.hidden_dim = hidden_dim
        self.gru_model = nn.GRU(word_embedding_dim, 
                                self.hidden_dim, 
                                batch_first=True, 
                                dropout=dropout_prob
                            )
        self.DropLayer = nn.Dropout(dropout_prob)
#         self.PoolLayer = pooling_layer(word_embedding_dim, sent_embedding_dim) 
        
    def forward(self, x_emb): 
        """
        input_x: [batchsize, max_seq_len, sentence_embedding_dim] 
        x_emb: [batchsize, max_seq_len, 1, embedding_dim]
        x_len: [batchsize]
        init_states: [batchsize, hidden_dim]
        """
        batchsize, max_seq_len, _ , emb_dim = x_emb.shape
        init_states = torch.zeros([1, batchsize, self.hidden_dim], dtype=torch.float32).cuda()
        pool_feature = x_emb.reshape(
                [-1, max_seq_len, emb_dim]
        )
        try:
            df_outputs, df_last_state = self.gru_model(pool_feature, init_states)
        except:
            print("Error:", pool_feature.shape, init_states.shape)
            raise
        # hidden_outs = [df_outputs[i][:x_len[i]] for i in range(batchsize)]
        # final_outs = [df_outputs[i][x_len[i]-1] for i in range(batchsize)]
        # return hidden_outs, final_outs
        return df_outputs


class CM_Model_V1(nn.Module):
    def __init__(self, hidden_dim, action_num):
        super(CM_Model_V1, self).__init__()
        self.hidden_dim = hidden_dim
        self.action_num = action_num
        self.DenseLayer = nn.Linear(self.hidden_dim, 64)
        self.Classifier = nn.Linear(64, self.action_num)
        
    def forward(self, rdm_state):
        """
        rdm_state: [batchsize, hidden_dim]
        """
        batchsize, hidden_dim = rdm_state.shape
        rl_h1 = nn.functional.relu(
            self.DenseLayer(
                rdm_state
            )
        )
        stopScore = self.Classifier(rl_h1)
        isStop = stopScore.argmax(axis=1)
        return stopScore, isStop

class CM_Model(nn.Module):
    def __init__(self, sentence_embedding_dim, hidden_dim, action_num):
        super(CM_Model, self).__init__()
        self.sentence_embedding_dim = sentence_embedding_dim
        self.hidden_dim = hidden_dim
        self.action_num = action_num
        self.DenseLayer = nn.Linear(self.hidden_dim, 64)
        self.Classifier = nn.Linear(64, self.action_num)
        
    def forward(self, rdm_model, rl_input, rl_state):
        """
        rl_input: [batchsize, max_word_num, sentence_embedding_dim]
        rl_state: [1, batchsize, hidden_dim]
        """
        assert(rl_input.ndim==3)
        batchsize, max_word_num, embedding_dim = rl_input.shape
        rl_output, rl_new_state = rdm_model.gru_model(
                                            rl_input, 
                                            rl_state
                                        )
        rl_h1 = nn.functional.relu(
            self.DenseLayer(
                rl_output.reshape(
                    [len(rl_input), self.hidden_dim]
                )
            )
        )
        stopScore = self.Classifier(rl_h1)
        isStop = stopScore.argmax(axis=1)
        return stopScore, isStop, rl_new_state


def layer2seq(bert, layer, cuda=False):
    if cuda:
        outs = [bert( torch.tensor([input_]).cuda())
                for input_ in layer]   
    else: 
        outs = [bert( torch.tensor([input_]))
                    for input_ in layer]
    states = [item[1] for item in outs]
    return rnn_utils.pad_sequence(states, batch_first=True)

def Word_ids2SeqStates(word_ids, bert, ndim, cuda=False):
    assert(ndim == 3)
    if cuda:
        embedding = [layer2seq(bert, layer, cuda) for layer in word_ids]
    else:
        embedding = [layer2seq(bert, layer) for layer in word_ids]
    return padding_sequence(embedding)


# In[4]:


def Count_Accs(ylabel, preds):
    correct_preds = np.array(
        [1 if y1==y2 else 0 
        for (y1, y2) in zip(ylabel, preds)]
    )
    y_idxs = [idx if yl >0 else idx - len(ylabel) 
            for (idx, yl) in enumerate(ylabel)]
    pos_idxs = list(filter(lambda x: x >= 0, y_idxs))
    neg_idxs = list(filter(lambda x: x < 0, y_idxs))
    acc = sum(correct_preds) / (1.0 * len(ylabel))
    if len(pos_idxs) > 0:
        pos_acc = sum(correct_preds[pos_idxs])/(1.0*len(pos_idxs))
    else:
        pos_acc = 0
    if len(neg_idxs) > 0:
        neg_acc = sum(correct_preds[neg_idxs])/(1.0*len(neg_idxs))
    else:
        neg_acc = 0
    return acc, pos_acc, neg_acc, y_idxs, pos_idxs, neg_idxs, correct_preds

def Loss_Fn(ylabel, pred_scores):
    diff = ((ylabel - pred_scores)*(ylabel - pred_scores)).mean(axis=1)
#     pos_neg = (1.0*sum(ylabel.argmax(axis=1)))/(1.0*(len(ylabel) - sum(ylabel.argmax(axis=1))))
    pos_neg = 0
    if pos_neg > 0:
        print("unbalanced data")
        weight = torch.ones(len(ylabel)).cuda() + (ylabel.argmax(axis=1).to(torch.float32)/(1.0*pos_neg)) - ylabel.argmax(axis=1).to(torch.float32)
        return (weight *diff).mean()
    else:
        print("totally unbalanced data")
        return diff.mean()


# In[5]:
def rdm_loss(x, y, x_len, bert, rdm_model, rdm_classifier, loss_fn):
    x_emb = Word_ids2SeqStates(x, bert, 3, cuda=True) 
    batchsize, max_seq_len, max_sent_len, emb_dim = x_emb.shape
    init_states = torch.zeros([1, batchsize, rdm_model.hidden_dim]).cuda()
    rdm_hiddens, rdm_outs = rdm_model(x_emb, x_len, init_states)
    rdm_scores = rdm_classifier(
        torch.cat(
            rdm_outs # a list of tensor, where the ndim of tensor is 1 and the shape of tensor is [hidden_size]
        ).reshape(
            [-1, rdm_model.hidden_dim]
        )
    )
    rdm_preds = rdm_scores.argmax(axis=1)
    y_label = y.argmax(axis=1)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, rdm_preds)
    loss = loss_fn(rdm_scores, torch.tensor(y_label).cuda())
    return loss, acc


def TrainRDMModel(rdm_model, bert, rdm_classifier, 
                    tokenizer, t_steps, new_data_len=[], logger=None, 
                        log_dir="RDMBertTrain"):
    batch_size = 20 
    max_gpu_batch = 5 #cannot load a larger batch into the limited memory, but we could  accumulates grads
    assert(batch_size%max_gpu_batch == 0)
    sum_loss = 0.0
    sum_acc = 0.0
    t_acc = 0.9
    ret_acc = 0.0
    init_states = torch.zeros([1, 5, rdm_model.hidden_dim], dtype=torch.float32).cuda()
    weight = torch.tensor([2.0, 1.0], dtype=torch.float32).cuda()
    loss_fn = nn.CrossEntropyLoss(weight=weight)
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-5},
                                {'params': rdm_classifier.parameters(), 'lr': 5e-5},
                                {'params': rdm_model.parameters(), 'lr': 5e-5}
                             ]
    )
    
    writer = SummaryWriter(log_dir)
    acc_l = np.zeros(int(batch_size/max_gpu_batch))
    loss_l = np.zeros(int(batch_size/max_gpu_batch))
    for step in range(t_steps):
        optim.zero_grad()
        for j in range(int(batch_size/max_gpu_batch)):
            if len(new_data_len) > 0:
                x, x_len, y = get_df_batch(step, max_gpu_batch, new_data_len, tokenizer=tokenizer)
            else:
                x, x_len, y = get_df_batch(step, max_gpu_batch, tokenizer=tokenizer)
                
            loss, acc = rdm_loss(x, y, bert, rdm_model, rdm_classifier, loss_fn)
            loss.backward()
            loss_l[j] = loss
            acc_l[j] = acc
            
        optim.step()        
        writer.add_scalar('Train Loss', loss_l.mean(), step)
        writer.add_scalar('Train Accuracy', acc_l.mean(), step)

        sum_loss += loss_l.mean()
        sum_acc += acc_l.mean()
        

        if step % 10 == 9:
            sum_loss = sum_loss / 10
            sum_acc = sum_acc / 10
            print('%3d | %d , train_loss/accuracy = %6.8f/%6.7f'             % (step, t_steps, 
                sum_loss, sum_acc,
                ))
            logger.info('%3d | %d , train_loss/accuracy = %6.8f/%6.7f'             % (step, t_steps, 
                sum_loss, sum_acc,
                ))
            if step%500 == 499:
                rdm_save_as = '/home/hadoop/ERD/%s/rdmModel_epoch%03d.pkl'                                    % (log_dir, step/500, sum_acc)
                torch.save(
                    {
                        "bert":bert.state_dict(),
                        "sentiModel":sentiModel.state_dict(),
                        "rmdModel":rdm_model.state_dict(),
                        "rdm_classifier": rdm_classifierdm.state_dict()
                    },
                    rdm_save_as
                )
#                 rdm_model, bert, sentiModel, rdm_classifier
            if sum_acc > t_acc:
                break
            sum_acc = 0.0
            sum_loss = 0.0

    print(get_curtime() + " Train df Model End.")
    logger.info(get_curtime() + " Train df Model End.")
    return ret_acc


# In[6]:


def TrainCMModel(bert, rdm_model, rdm_classifier, cm_model, tokenizer, log_dir, logger, FLAGS):
    batch_size = 20
    t_acc = 0.9
    ids = np.array(range(batch_size), dtype=np.int32)
    seq_states = np.zeros([batch_size], dtype=np.int32)
    isStop = np.zeros([batch_size], dtype=np.int32)
    max_id = batch_size
    df_init_states = torch.zeros([1, batch_size, FLAGS.hidden_dim], dtype=torch.float32).cuda()
    state = df_init_states
    D = deque()
    ssq = []
    print("in RL the begining")
    rdm_optim = torch.optim.Adagrad([
                            {'params': bert.parameters(), 'lr':1e-3},
    #                                 {'params': rdm_classifier.parameters(), 'lr': 5e-2},
                            {'params': rdm_model.parameters(), 'lr': 5e-2},
                            {'params': sentiModel.parameters(), 'lr': 1e-2}
                         ],
                            weight_decay = 0.2
    )
    rl_optim = torch.optim.Adam([{'params':cm_model.parameters(), 'lr':1e-3}])
    # get_new_len(sess, mm)
    data_ID = get_data_ID()

    if len(data_ID) % batch_size == 0: # the total number of events
        flags = int(len(data_ID) / FLAGS.batch_size)
    else:
        flags = int(len(data_ID) / FLAGS.batch_size) + 1
    for i in range(flags):
        with torch.no_grad():
            x, x_len, y = get_df_batch(i, batch_size, tokenizer=tokenizer)
            x_emb = Word_ids2SeqStates(x, bert, 3, cuda=True) 
            batchsize, max_seq_len, max_sent_len,                                     emb_dim = x_emb.shape
            sent_feature = sentiModel( 
                x_emb.reshape(
                    [-1, max_sent_len, emb_dim]
                ) 
            ).reshape(
                [batchsize, max_seq_len, -1]
            )
            rdm_hiddens, rdm_outs = rdm_model(sent_feature, x_len, df_init_states)
        #         t_ssq = sess.run(rdm_train.out_seq, feed_dic)# t_ssq = [batchsize, max_seq, scores]
            print("batch %d"%i)
            if len(ssq) > 0:
                ssq.extend([rdm_classifier(h) for h in rdm_hiddens])
            else:
                ssq = [rdm_classifier(h) for h in rdm_hiddens]

    print(get_curtime() + " Now Start RL training ...")
    counter = 0
    sum_rw = 0.0 # sum of rewards

    data_len = get_data_len()

    while True:
        if counter > FLAGS.OBSERVE:
            sum_rw += np.mean(rw)
            if counter % 200 == 0:
                sum_rw = sum_rw / 2000
                print( get_curtime() + " Step: " + str(step) 
                       + " REWARD IS " + str(sum_rw) 
                     )
                logger.info( get_curtime() + 
                             " Step: " + str(step) + 
                            " REWARD IS " + str(sum_rw)
                           )
                if sum_rw > t_rw:
                    print("Retch The Target Reward")
                    logger.info("Retch The Target Reward")
                    break
                if counter > t_steps:
                    print("Retch The Target Steps")
                    logger.info("Retch The Target Steps")
                    break
                sum_rw = 0.0
            s_state, s_x, s_isStop, s_rw = get_RL_Train_batch(D, FLAGS)
            stopScore, isStop, rl_new_state = cm_model(rdm_model, sentiModel, s_x, s_state)
            out_action = (stopScore*s_isStop).sum(axis=1)
            rl_cost = torch.mean((s_rw - out_action)*(s_rw - out_action))
            rl_cost.backward()
            rl_optim.step()

        input_x, input_y, ids, seq_states, max_id = get_rl_batch(ids, seq_states, isStop, max_id, 0, FLAGS, tokenizer=tokenizer)
        with torch.no_grad():
            x_emb = layer2seq(bert, input_x, cuda=True)
            batchsize, max_sent_len, emb_dim = x_emb.shape
            mss, isStop, mNewState = cm_model(rdm_model, sentiModel, x_emb, state)

        for j in range(FLAGS.batch_size):
            if random.random() < FLAGS.random_rate:
    #             isStop[j] = np.argmax(np.random.rand(2))
                isStop[j] = int(torch.rand(2).argmax())
            if seq_states[j] == data_len[ids[j]]:
                isStop[j] = 1

        # eval
        rw = get_reward(isStop, mss, ssq, ids, seq_states)

        for j in range(FLAGS.batch_size):
            D.append((state[0][j], input_x[j], isStop[j], rw[j]))
            if len(D) > FLAGS.max_memory:
                D.popleft()

        state = mNewState
        for j in range(FLAGS.batch_size):
            if isStop[j] == 1:
                # init_states = np.zeros([FLAGS.batch_size, FLAGS.hidden_dim], dtype=np.float32)
                # feed_dic = {rl_model.init_states: init_states}
                # state[j] = sess.run(rl_model.df_state, feed_dic)
    #             state[j] = np.zeros([FLAGS.hidden_dim], dtype=np.float32)
                state[0][j].fill_(0)
        counter += 1.0

## 联合学习部分


### 工具函数

In [5]:
def words2tensor(bert, layer, cuda=False):
    if cuda:
        outs = [bert( torch.tensor([input_]).cuda())
                for input_ in layer]   
    else: 
        outs = [bert( torch.tensor([input_]))
                    for input_ in layer]
    states = [item[0][0] for item in outs]
    return rnn_utils.pad_sequence(states, batch_first=True)


> normalized the data for unbalanced learning:
```python
weights = WeightsForUmbalanced(emoReader.label)
weights
```

In [6]:

def rdm_data2bert_tensors(data_X, cuda):
    def padding_sent_list(sent_list):
        sent_len = [len(sent) for sent in sent_list]
        max_sent_len = max(sent_len)
        sent_padding = torch.zeros([len(sent_list), max_sent_len], dtype=torch.int64)
        attn_mask = torch.ones_like(sent_padding)
        for i, sent in enumerate(sent_list):
            sent_padding[i][:len(sent)] = torch.tensor(sent, dtype=torch.int32)
            attn_mask[i][len(sent):].fill_(0)
        return sent_padding, attn_mask
    sent_list = []
    [sent_list.extend(seq) for seq in data_X]
    seq_len = [len(seq) for seq in data_X]
    sent_tensors, attn_mask = padding_sent_list(sent_list)
    if cuda:
        sent_tensors = sent_tensors.cuda()
        attn_mask = attn_mask.cuda()
    return sent_tensors, attn_mask, seq_len

def sent_list2bert_tensors(sent_list, cuda):
    sent_len = [len(sent) for sent in sent_list]
    max_sent_len = max(sent_len)
    sent_padding = torch.zeros([len(sent_list), max_sent_len], dtype=torch.int64)
    attn_mask = torch.ones_like(sent_padding)
    for i, sent in enumerate(sent_list):
        sent_padding[i][:len(sent)] = torch.tensor(sent, dtype=torch.int32)
        attn_mask[i][len(sent):].fill_(0)
    if cuda:
        sent_padding = sent_padding.cuda()
        attn_mask = attn_mask.cuda()
    return sent_padding, attn_mask

In [7]:
def rdm_data2bert_tensors(data_X, cuda):
    def padding_sent_list(sent_list):
        sent_len = [len(sent) for sent in sent_list]
        max_sent_len = max(sent_len)
        sent_padding = torch.zeros([len(sent_list), max_sent_len], dtype=torch.int64)
        attn_mask = torch.ones_like(sent_padding)
        for i, sent in enumerate(sent_list):
            sent_padding[i][:len(sent)] = torch.tensor(sent, dtype=torch.int32)
            attn_mask[i][len(sent):].fill_(0)
        return sent_padding, attn_mask
    sent_list = []
    [sent_list.extend(seq) for seq in data_X]
    seq_len = [len(seq) for seq in data_X]
    sent_tensors, attn_mask = padding_sent_list(sent_list)
    if cuda:
        sent_tensors = sent_tensors.cuda()
        attn_mask = attn_mask.cuda()
    return sent_tensors, attn_mask, seq_len

def senti_data2bert_tensors(sent_list, cuda):
    sent_len = [len(sent) for sent in sent_list]
    max_sent_len = max(sent_len)
    sent_padding = torch.zeros([len(sent_list), max_sent_len], dtype=torch.int64)
    attn_mask = torch.ones_like(sent_padding)
    input_mask = torch.zeros
    for i, sent in enumerate(sent_list):
        sent_padding[i][:len(sent)] = torch.tensor(sent, dtype=torch.int32)
        attn_mask[i][len(sent):].fill_(0)
    if cuda:
        sent_padding = sent_padding.cuda()
        attn_mask = attn_mask.cuda()
    return sent_padding, attn_mask

In [8]:
def WeightsForUmbalanced(data_label):
    _, _, labels = data_label.shape
    label_cnt = data_label.reshape([-1, labels]).sum(axis=0)
    weights = 1.0/label_cnt
    normalized_weights = weights/sum(weights)
    return normalized_weights

### 训练各个任务

##### 使用单个分类器进行训练

In [8]:
def emo_cls_train(emotion_reader, val_reader, bert, transformer, 
                  task_embedding, emotion_classifier,
                  train_epochs, 
                  log_dir,
                  cuda=False
                 ):
    optim = torch.optim.Adam([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':1e-5},
                                {'params': transformer.parameters(), 'lr': 1e-5},
                                {'params': emotion_classifier.parameters(), 'lr': 1e-5}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    writer = SummaryWriter(log_dir)
    e_weight = WeightsForUmbalanced(emotion_reader.label) 
    e_weight = torch.tensor(e_weight, dtype= torch.float32) if not cuda else torch.tensor(e_weight, dtype=torch.float32).cuda()
    loss_fn = nn.CrossEntropyLoss(weight=e_weight)
    emo_task_id = torch.tensor([2]).cuda() if cuda else torch.tensor([2])
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for xem, yem, lem in emotion_reader.iter():
            bs, l_cnt = yem.shape
            emo_tensors, emo_mask = senti_data2bert_tensors(xem, cuda)
            xem_embs, _ = bert(emo_tensors, attention_mask = emo_mask)
            tensors = xem_embs + task_embedding(emo_task_id)
            emo_feature = transformer(tensors, attention_mask = emo_mask)
            cls_feature = emo_feature[0][:, 0]
            emo_scores = emotion_classifier(cls_feature)
            emo_label = torch.tensor(yem).argmax(axis=1) if not cuda else torch.tensor(yem).argmax(axis=1).cuda()
            emo_loss = loss_fn(emo_scores, emo_label)
            emo_loss.backward()
            optim.step()
            torch.cuda.empty_cache()
            losses[int(step%10)] = emo_loss.cpu()
            emo_acc = accuracy_score(yem.argmax(axis=1), emo_scores.argmax(axis=1).cpu())
            accs[int(step%10)] = emo_acc
            writer.add_scalar('Train Loss', emo_loss.cpu(), step)
            writer.add_scalar('Train Accuracy', emo_acc, step)
            print("step:%d | loss/acc = %.3f/%.3f"%(step, emo_loss, emo_acc))
            if step %10 == 9:
                print('emotion task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )       
            step += 1 
        with torch.no_grad():
            batchs, bs, l_cnt = val_reader.label.shape
            losses = np.zeros(batchs)
            acces = np.zeros(batchs)
            it = 0
            for xem, yem, lem in val_reader.iter():
                emo_tensors, emo_mask = senti_data2bert_tensors(xem, cuda)
                xem_embs, _ = bert(emo_tensors, attention_mask = emo_mask)
                tensors = xem_embs + task_embedding(emo_task_id)
                emo_feature = transformer(tensors, attention_mask = emo_mask)
                cls_feature = emo_feature[0][:, 0]
                emo_scores = emotion_classifier(cls_feature)
                emo_label = torch.tensor(yem).argmax(axis=1) if not cuda else torch.tensor(yem).argmax(axis=1).cuda()
                emo_loss = loss_fn(emo_scores, emo_label)
                torch.cuda.empty_cache()
                emo_acc = accuracy_score(yem.argmax(axis=1), emo_scores.argmax(axis=1).cpu()) 
                losses[it] = emo_loss
                acces[it] = emo_acc
                it += 1
            writer.add_scalar('valid Loss', losses.mean(), epoch)
            writer.add_scalar('valid Accuracy', acces.mean(), epoch)
        cls_save_as = './%s/EmoModel_epoch%03d.pkl'% (log_dir, epoch)
        torch.save(
                            {
                                "bert":bert.state_dict(),
                                "transformer":transformer.state_dict(),
                                "task_embedding":task_embedding.state_dict(),
                                "emotion_classifier": emotion_classifier.state_dict()
                            },
                            cls_save_as
                        )   

##### 使用多个分类器进行训练

In [9]:
def emo_cls_train(emotion_reader, bert, transformer, 
                  task_embedding, emotion_classifiers,
                  train_epochs, 
                  cuda=False
                 ):
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':1e-6},
                                {'params': transformer.parameters(), 'lr': 1e-6},
                                {'params': emotion_classifiers.parameters(), 'lr': 1e-6}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    label_cnt = emoReader.label.reshape([-1, 6]).sum(axis=0)
    sum_cnt = sum(label_cnt)
    emo_weights_bi = [torch.tensor([(label_cnt[i]*1.0)/sum_cnt, (sum_cnt - label_cnt[i])*1.0/sum_cnt]) for i in range(emo_weights.shape[-1])]
    loss_list = nn.ModuleList([nn.CrossEntropyLoss(weight=e_weight.cuda()) for e_weight in emo_weights_bi])
    emo_task_id = torch.tensor([2]).cuda() if cuda else torch.tensor([2])
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for xem, yem, lem in emotion_reader.iter():
            bs, l_cnt = yem.shape
            emo_tensors, emo_mask = senti_data2bert_tensors(xem, cuda)
            xem_embs, _ = bert(emo_tensors, attention_mask = emo_mask)

            tensors = xem_embs + task_embedding(emo_task_id)
            emo_feature = transformer(tensors.transpose(0, 1)).transpose(0, 1)
            cls_feature = emo_feature.max(axis=1)[0]
            emo_scores = [emotion_classifier(cls_feature) for emotion_classifier in emotion_classifiers]

            labels = torch.zeros([bs, l_cnt, 2])
            y = yem.astype(np.int32)
            [[labels[idx][jdx][y_i].fill_(1.0) for jdx, y_i in enumerate(ylabel) ] for idx, ylabel in enumerate(y)]
            labels = labels.transpose(0, 1).cuda() if cuda else labels.transpose(0, 1)
            emo_loss = sum([loss_list[i](emo_scores[i], labels[i].argmax(axis=1)) for i in range(l_cnt)])
            emo_loss.backward()
            optim.step()
            torch.cuda.empty_cache()
            losses[int(step%10)] = emo_loss.cpu()
            preds = torch.cat([tensor[:, 1] for tensor in emo_scores]).reshape(l_cnt, bs).max(axis=0)[1]
            emo_acc = accuracy_score(y.argmax(axis=1), preds.cpu())

            accs[int(step%10)] = emo_acc
            print("step:%d | loss/acc = %.3f/%.3f"%(step, emo_loss, emo_acc))
            if step %10 == 9:
                print('emotion task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )       
            step += 1 

import os

os.path.exists("./RDMBertTrain/cached_ssq.pkl")

#### 训练subjective classification　任务

In [9]:
def subj_cls_train(subj_reader, valid_reader, 
                   bert, task_embedding, subj_pooler, subj_classifier, 
                   train_epochs, cuda=False, log_dir="SubjRDM"
                  ):
    subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_reader.label
            ),
            dtype = torch.float32
    )
    subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights) if not cuda else nn.CrossEntropyLoss(weight=subj_weights.cuda())

    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-5},
                                {'params': subj_pooler.parameters(), 'lr': 5e-3},
                                {'params': task_embedding.parameters(), 'lr':5e-3},
                                {'params': subj_classifier.parameters(), 'lr': 5e-3}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    
    writer = SummaryWriter(log_dir)
    
    subj_task_id = torch.tensor([1]).cuda() if cuda else torch.tensor([1])
    best_valid_acc = 0.0
    
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for xsj, ysj, lsj in subj_reader.iter():
            sent_tensors, sent_mask = senti_data2bert_tensors(xsj, cuda)
            xsj_embs, _ = bert(sent_tensors, attention_mask = sent_mask, task_emb=task_embedding(subj_task_id))
            cls_feature = subj_pooler(xsj_embs[:, 0])
            subj_scores = subj_cls(cls_feature)
            y_label = torch.tensor(ysj.argmax(axis=1)).cuda() if cuda else torch.tensor(ysj.argmax(axis=1))
            sj_loss = subj_loss_fn(subj_scores, y_label)
            sj_acc = accuracy_score(ysj.argmax(axis=1), subj_scores.cpu().argmax(axis=1))
            optim.zero_grad()
            sj_loss.backward()
            optim.step()
            torch.cuda.empty_cache()
            
            losses[int(step%10)] = sj_loss.cpu()
            accs[int(step%10)] = sj_acc
            
            print("step:%d | loss/acc = %.3f/%.3f"%(step, sj_loss, sj_acc))
            writer.add_scalar('Train Loss', sj_loss.cpu(), step)
            writer.add_scalar('Train Accuracy', sj_acc, step)
            if step %10 == 9:
                print('subjective task: %6d: [%5d/%5d], subj_loss/subj_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )
            step += 1 
        
        with torch.no_grad():
            bs_cnt, bs, l_cnt = valid_reader.label.shape
            preds = []
            losses = np.zeros(bs_cnt)
            it = 0
            for xsj, ysj, lsj in valid_reader.iter():
                sent_tensors, sent_mask = senti_data2bert_tensors(xsj, cuda)
                xsj_embs, _ = bert(sent_tensors, attention_mask = sent_mask, task_emb=task_embedding(subj_task_id))
                cls_feature = subj_pooler(xsj_embs[:, 0])
                subj_scores = subj_cls(cls_feature)
                y_label = torch.tensor(ysj.argmax(axis=1)).cuda() if cuda else torch.tensor(ysj.argmax(axis=1))
                sj_loss = subj_loss_fn(subj_scores, y_label)
                sj_acc = accuracy_score(ysj.argmax(axis=1), subj_scores.cpu().argmax(axis=1))
                losses[it] = sj_loss
                preds.append(subj_scores)
                torch.cuda.empty_cache()
                it += 1
            val_preds = torch.cat(preds).cpu()
            val_acc = accuracy_score(valid_reader.label.reshape(bs_cnt*bs, l_cnt).argmax(axis=1), val_preds.argmax(axis=1))
            val_loss = losses.mean()
            writer.add_scalar('valid Loss', val_loss, epoch)
            writer.add_scalar('valid Accuracy', val_acc, epoch)
            print("valid loss/acc: %.6f/%.6f:"%(val_loss, val_acc))

        if val_acc > best_valid_acc:
            best_valid_acc = val_acc
            print("best_valid_acc:", best_valid_acc)
            subj_save_as = './%s/subj_best_Model.pkl'% (log_dir)
            torch.save(
                {
                    "bert":bert.state_dict(),
                    "task_embedding":task_embedding.state_dict(),
                    "subj_pooler":subj_pooler.state_dict(),
                    "subj_classifier": subj_classifier.state_dict(),
                },
                subj_save_as
            )

#### 训练sentiment analysis任务

In [10]:
def senti_cls_train(senti_reader, valid_reader,
                    bert, task_embedding, senti_pooler, senti_classifier,
                    train_epochs, cuda=False, log_dir="SentiRDM"
                   ):
    optim = torch.optim.Adam([
                                {'params': bert.parameters(), 'lr':1e-5},
                                {'params': task_embedding.parameters(), 'lr':1e-4},
                                {'params': senti_pooler.parameters(), 'lr': 1e-4},
                                {'params': senti_classifier.parameters(), 'lr': 1e-4}
                             ]
    )

    #  senti loss graph 
    #-----------------------------------------------------------
    senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_reader.label
            ),
            dtype=torch.float32
    )
    senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda()) if cuda else nn.CrossEntropyLoss(weight=senti_weights)
    senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()
    #-------------------------------------------------------------    
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    best_valid_acc = 0
    writer = SummaryWriter(log_dir)
    
    task_emb = task_embedding(torch.tensor([0]).cuda()) if cuda else task_embedding(torch.tensor([0]))
    
    optim.zero_grad()
    
    step = 0
    for epoch in range(train_epochs):
        for xst, yst, lst in senti_reader.iter():
            sent_tensors, sent_mask = senti_data2bert_tensors(xst, cuda)
            xst_embs, _ = bert(sent_tensors, attention_mask = sent_mask, task_emb=task_embedding(senti_task_id))
            cls_feature = senti_pooler(xst_embs[:, 0])
            senti_scores = senti_classifier(cls_feature)
            y_label = torch.tensor(yst.argmax(axis=1)).cuda() if cuda else torch.tensor(yst.argmax(axis=1))
            st_loss = senti_loss_fn(senti_scores, y_label)
            st_acc = accuracy_score(yst.argmax(axis=1), senti_scores.cpu().argmax(axis=1))
            optim.zero_grad()
            st_loss.backward()
            torch.cuda.empty_cache()
            optim.step()
            losses[int(step%10)] = st_loss.cpu()
            accs[int(step%10)] = st_acc
            print("step:%d | loss/acc = %.3f/%.3f"%(step, st_loss, st_acc))
            writer.add_scalar('Train Loss', st_loss.cpu(), step)
            writer.add_scalar('Train Accuracy', st_acc, step)
            if step %10 == 9:
                print('sentiment task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         ) 
            step += 1
            
        with torch.no_grad():
            bs_cnt, bs, l_cnt = valid_reader.label.shape
            preds = []
            losses = np.zeros(bs_cnt)
            it = 0
            for xst, yst, lst in valid_reader.iter():
                sent_tensors, sent_mask = senti_data2bert_tensors(xst, cuda)
                xst_embs, _ = bert(sent_tensors, attention_mask = sent_mask, task_emb=task_embedding(senti_task_id))
                cls_feature = senti_pooler(xst_embs[:, 0])
                senti_scores = senti_classifier(cls_feature)
                y_label = torch.tensor(yst.argmax(axis=1)).cuda() if cuda else torch.tensor(yst.argmax(axis=1))
                st_loss = senti_loss_fn(senti_scores, y_label)
                st_acc = accuracy_score(yst.argmax(axis=1), senti_scores.cpu().argmax(axis=1))
                losses[it] = st_loss
                preds.append(senti_scores)
                torch.cuda.empty_cache()
            val_preds = torch.cat(preds).cpu()
            val_acc = accuracy_score(valid_reader.label.reshape(bs_cnt*bs, l_cnt).argmax(axis=1), val_preds.argmax(axis=1))
            val_loss = losses.mean()
            writer.add_scalar('valid Loss', val_loss, epoch)
            writer.add_scalar('valid Accuracy', val_acc, epoch)
            print("valid loss/acc: %.6f/%.6f:"%(val_loss, val_acc))

        if val_acc > best_valid_acc:
            best_valid_acc = val_acc
            print("best_valid_acc:", best_valid_acc)
            senti_save_as = './%s/senti_best_Model.pkl'% (log_dir)
            torch.save(
                {
                    "bert":bert.state_dict(),
                    "task_embedding":task_embedding.state_dict(),
                    "senti_pooler":senti_pooler.state_dict(),
                    "senti_classifier": senti_classifier.state_dict()
                },
                senti_save_as
            )


### 联合训练函数

#### 三人舞
情感任务，主观客观性任务，　情绪分析任务，　三个共同训练的函数

In [13]:
def JointLearning(senti_reader, subj_reader, emotion_reader, 
                  bert, transformer, task_embedding, 
                  senti_classifier, subj_classifier, emotion_classifier,
                  cuda=False
                 ):
    
    #stage 3: deploy the trainning on the emotion classification task
#     emo_weights = torch.tensor(
#             WeightsForUmbalanced(
#                 emotion_reader.label
#             ),
#             dtype = torch.float32
#     )
#     emo_loss_fn = nn.CrossEntropyLoss(weight=emo_weights) if not cuda else nn.CrossEntropyLoss(weight=emo_weights.cuda())
#     emo_cls_train(
#         emotion_reader, 
#         bert, 
#         transformer, 
#         task_embedding, 
#         emotion_classifier, 
#         2, 
#         emo_loss_fn,
#         cuda
#     )
#     emotion_reader.reset_batchsize(5)    
    
    # stage 1: deploy the trainning on the senti classification task
    senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_reader.label
            ),
            dtype=torch.float32
    )
    senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda()) if cuda else nn.CrossEntropyLoss(weight=senti_weights)
    senti_cls_train(
                    senti_reader, 
                    bert, 
                    transformer, 
                    task_embedding, 
                    senti_classifier, 
                    2, 
                    senti_loss_fn,
                    cuda
                   )
    #stage 2: deploy the trainning on the subj classification task
    subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_reader.label
            ),
            dtype = torch.float32
    )
    subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights) if not cuda else nn.CrossEntropyLoss(weight=subj_weights.cuda())
    subj_cls_train(
                   subj_reader, 
                   bert, 
                   transformer, 
                   task_embedding, 
                   subj_classifier, 
                   2, 
                   subj_loss_fn,
                   cuda
                  )
    
    joint_model_save_as = '/home/hadoop/ERD/MTLTrain/PreTrModel_epoch.pkl'
    torch.save(
        {
            "bert":bert.state_dict(),
            "transformer":transformer.state_dict(),
            "task_embedding":task_embedding.state_dict(),
            "senti_classifier": senti_classifier.state_dict(),
            "subj_classifier": subj_classifier.state_dict(),
#             "emotion_classifier": emotion_classifier.state_dict()
        },
        joint_model_save_as
    )
    print("saved pretrained model!")

    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-7},
                                {'params': task_embedding.parameters(), 'lr':1e-6},
                                {'params': transformer.parameters(), 'lr': 1e-6},
                                {'params': senti_classifier.parameters(), 'lr': 1e-6},
                                {'params': subj_classifier.parameters(), 'lr':1e-6},
                                {'params': emotion_classifier.parameters(), 'lr':1e-6}
                             ]
    )
    
    max_epoch = 100
    
    losses = np.zeros([3, 10]) 
    accs = np.zeros([3, 10])
    # [[senti_loss_1, ..., senti_loss_10], [subj_loss_1, ..., subj_loss_10], [emo_loss_1, ..., emo_loss_10]] 
    senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()
    subj_task_id = torch.tensor([1]) if not cuda else torch.tensor([1]).cuda()
#     emo_task_id = torch.tensor([2]) if not cuda else torch.tensor([2]).cuda()
    
    loss_weight = torch.tensor([0.333, 0.333, 0.333]) if not cuda else torch.tensor([0.333, 0.333, 0.333]).cuda()
    
    batchs = min(senti_reader.label.shape[0], subj_reader.label.shape[0], emotion_reader.label.shape[0])
    optim.zero_grad()
    for epoch in range(max_epoch):
        step = 0
        for ((xst, yst, lst), (xsj, ysj, lsj), (xem, yem, lem)) in zip(senti_reader.iter(), subj_reader.iter(), emotion_reader.iter()):
            st_loss, st_acc = senti_loss(xst, yst, lst, 
                                    bert, transformer, task_embedding(senti_task_id),
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
#             st_loss.backward()
            MTL_Loss = st_loss*loss_weight[0]
    
            losses[0][step%10] = st_loss.tolist()
            accs[0][step%10] = st_acc
             
            sj_loss, sj_acc = subj_loss(xsj, ysj, lsj,
                                    bert, transformer, task_embedding(subj_task_id),
                                    subj_classifier, subj_loss_fn,
                                    cuda
                                   )
#             sj_loss.backward()
            MTL_Loss += sj_loss*loss_weight[1]

            losses[1][step%10] = sj_loss.tolist()
            accs[1][step%10] = sj_acc
            
            emo_loss, emo_acc = emotion_loss(xem, yem, lem,
                                    bert, transformer, task_embedding(emo_task_id),
                                    emotion_classifier, emo_loss_fn,
                                    cuda
                                   )
#             emo_loss.backward()
            MTL_Loss += emo_loss*loss_weight[2]
    
            losses[2][step%10] = emo_loss.tolist()
            accs[2][step%10] = emo_acc
            
            MTL_Loss.backward()
            optim.step()
            optim.zero_grad()
            print("%6d|MTL_Loss:%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f | emo_loss/emo_acc = %6.8f/%6.7f" % (
                                                                                                step, MTL_Loss,        
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean(),
                                                                                                losses[2].mean(), accs[2].mean()
            )
            )
            if step % 10 == 9:
                print('%6d: [%5d/%5d], MTL_Loss|%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f | emo_loss/emo_acc = %6.8f/%6.7f' % (
                                                                                                step,
                                                                                                epoch,max_epoch, MTL_Loss,
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean(),
                                                                                                losses[2].mean(), accs[2].mean()
                                                                                            )
                     )
                loss_weight = torch.tensor(
                                            (1.0/accs.mean(axis=1))/sum(1.0/accs.mean(axis=1)),
                                            dtype=torch.float32
                                )
                print("\n\n loss_weight:", loss_weight)
            step += 1
        joint_model_save_as = '/home/hadoop/ERD/MTLTrain/jointModel_epoch%03d.pkl'% (epoch)
        torch.save(
            {
                "bert":bert.state_dict(),
                "transformer":transformer.state_dict(),
                "task_embedding":task_embedding.state_dict(),
                "senti_classifier": senti_classifier.state_dict(),
                "subj_classifier": subj_classifier.state_dict(),
                "emotion_classifier": emotion_classifier.state_dict()
            },
            joint_model_save_as
        )

#### 双人舞
情感分析和主观客观性任务联合训练的函数

In [14]:
def SentiSubjLearning(senti_reader, subj_reader, 
                  bert, transformer, task_embedding, 
                  senti_classifier, subj_classifier,
                  cuda=False
                 ):  
    
    # stage 1: deploy the trainning on the senti classification task
    senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_reader.label
            ),
            dtype=torch.float32
    )
    senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda()) if cuda else nn.CrossEntropyLoss(weight=senti_weights)
    #stage 2: deploy the trainning on the subj classification task
    subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_reader.label
            ),
            dtype = torch.float32
    )
    subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights) if not cuda else nn.CrossEntropyLoss(weight=subj_weights.cuda())

    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-7},
                                {'params': task_embedding.parameters(), 'lr':1e-6},
                                {'params': transformer.parameters(), 'lr': 1e-6},
                                {'params': senti_classifier.parameters(), 'lr': 1e-6},
                                {'params': subj_classifier.parameters(), 'lr':1e-6}
                             ]
    )
    
    max_epoch = 10
    
    losses = np.zeros([3, 10]) 
    accs = np.zeros([3, 10])
    # [[senti_loss_1, ..., senti_loss_10], [subj_loss_1, ..., subj_loss_10], [emo_loss_1, ..., emo_loss_10]] 
    senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()
    subj_task_id = torch.tensor([1]) if not cuda else torch.tensor([1]).cuda()
    
    loss_weight = torch.tensor([0.5, 0.5]) if not cuda else torch.tensor([0.5, 0.5]).cuda()
    
    batchs = min(senti_reader.label.shape[0], subj_reader.label.shape[0])
    optim.zero_grad()
    for epoch in range(10, max_epoch+10):
        step = 0
        for ((xst, yst, lst), (xsj, ysj, lsj)) in zip(senti_reader.iter(), subj_reader.iter() ):
            st_loss, st_acc = senti_loss(xst, yst, lst, 
                                    bert, transformer, task_embedding(senti_task_id),
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
            MTL_Loss = st_loss*loss_weight[0]
            losses[0][step%10] = st_loss.tolist()
            accs[0][step%10] = st_acc
             
            sj_loss, sj_acc = subj_loss(xsj, ysj, lsj,
                                    bert, transformer, task_embedding(subj_task_id),
                                    subj_classifier, subj_loss_fn,
                                    cuda
                                   )
            MTL_Loss += sj_loss*loss_weight[1]
            losses[1][step%10] = sj_loss.tolist()
            accs[1][step%10] = sj_acc
            
            MTL_Loss.backward()
            optim.step()
            optim.zero_grad()
            print("%6d %6d|MTL_Loss:%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f " % (
                                                                                                step, batchs, MTL_Loss,        
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean()
            )
            )
            if step % 10 == 9:
                print('%6d %6d: [%5d/%5d], MTL_Loss|%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f' % (
                                                                                                step, batchs, 
                                                                                                epoch,max_epoch, MTL_Loss,
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean()
                                                                                            )
                     )
                loss_weight = torch.tensor(
                                            (1.0/( accs.mean(axis=1)[:2] ) )/sum( 1.0/accs.mean(axis=1)[:2] ),
                                            dtype=torch.float32
                                )
                print("\n\n loss_weight:", loss_weight)
            step += 1
        joint_model_save_as = '/home/hadoop/ERD/MTLTrain/jointModel_epoch%03d.pkl'% (epoch)
        torch.save(
            {
                "bert":bert.state_dict(),
                "transformer":transformer.state_dict(),
                "task_embedding":task_embedding.state_dict(),
                "senti_classifier": senti_classifier.state_dict(),
                "subj_classifier": subj_classifier.state_dict()
            },
            joint_model_save_as
        )

#### 双任务增强
subj任务和sentiment任务来增强谣言检测模块

> 此时应当要能从Reader中随机地挑出一个batch出来训练

In [15]:
def MTLTrainRDMModel(rdm_model, bert, rdm_classifier,
                     transformer, task_embedding, senti_classifier, subj_classifier, 
                     sentiReader, subjReader, 
                    tokenizer, t_steps, new_data_len=[], logger=None, cuda=False, 
                        log_dir="RDMBertTrain"):
    batch_size = 10 
    max_gpu_batch = 2 #cannot load a larger batch into the limited memory, but we could  accumulates grads
    sentiReader.reset_batchsize(max_gpu_batch)
    subjReader.reset_batchsize(max_gpu_batch)
    
    assert(batch_size%max_gpu_batch == 0)
    
    sum_loss = np.zeros(4)
    sum_acc = np.zeros(3)
    
    t_acc = 0.9
    ret_acc = 0.0
    init_states = torch.zeros([1, 5, rdm_model.hidden_dim], dtype=torch.float32).cuda()
    
    senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()
    subj_task_id = torch.tensor([1]) if not cuda else torch.tensor([1]).cuda()
    
    weight = torch.tensor([2.0, 1.0], dtype=torch.float32).cuda()
    loss_fn = nn.CrossEntropyLoss(weight=weight)
    
    senti_weights = torch.tensor(
            WeightsForUmbalanced(
                sentiReader.label
            ),
            dtype=torch.float32
    )
    senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda()) if cuda else nn.CrossEntropyLoss(weight=senti_weights)
    
    subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subjReader.label
            ),
            dtype = torch.float32
    )
    subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights) if not cuda else nn.CrossEntropyLoss(weight=subj_weights.cuda())

    loss_weight = torch.tensor([0.8, 0.1, 0.1]) if not cuda else torch.tensor([0.8, 0.1, 0.1]).cuda()   
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-7},
                                {'params': rdm_classifier.parameters(), 'lr': 5e-5},
                                {'params': rdm_model.parameters(), 'lr': 5e-5},
                                {'params': task_embedding.parameters(), 'lr':1e-6},
                                {'params': transformer.parameters(), 'lr': 1e-6},
                                {'params': senti_classifier.parameters(), 'lr': 1e-6},
                                {'params': subj_classifier.parameters(), 'lr':1e-6}
                             ]
    )
    
    writer = SummaryWriter(log_dir)
    
    acc_tmp = np.zeros([3, int(batch_size/max_gpu_batch)])
    loss_tmp = np.zeros([4, int(batch_size/max_gpu_batch)])
    
    for step in range(t_steps):
        optim.zero_grad()
        for j in range(int(batch_size/max_gpu_batch)):
            if len(new_data_len) > 0:
                x, x_len, y = get_df_batch(step, max_gpu_batch, new_data_len, tokenizer=tokenizer)
            else:
                x, x_len, y = get_df_batch(step, max_gpu_batch, tokenizer=tokenizer) 
            loss, acc = rdm_loss(x, y, x_len, bert, rdm_model, rdm_classifier, loss_fn)
            MTL_Loss = loss*loss_weight[0]
            
            xst, yst, lst = sentiReader.sample()
            st_loss, st_acc = senti_loss(xst, yst, lst, 
                                    bert, transformer, task_embedding(senti_task_id),
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
            MTL_Loss += st_loss*loss_weight[1]
            
            xsj, ysj, lsj = subjReader.sample()
            sj_loss, sj_acc = subj_loss(xsj, ysj, lsj,
                                    bert, transformer, task_embedding(subj_task_id),
                                    subj_classifier, subj_loss_fn,
                                    cuda
                                   )
            MTL_Loss += sj_loss*loss_weight[2]
            
            MTL_Loss.backward()
            
            loss_tmp[:, j] = np.array([MTL_Loss, loss, st_loss, sj_loss])
            acc_tmp[:, j] = np.array([acc, st_acc, sj_acc])
            
        optim.step()        
        writer.add_scalar('Train Loss', loss_tmp[0].mean(), step)
        writer.add_scalar('Train Accuracy', acc_tmp[0].mean(), step)
        
        sum_acc += acc_tmp.mean(axis=1)
        sum_loss += loss_tmp.mean(axis=1)
        
        print("%6d %6d|MTL_Loss:%6.8f, rdm_loss/rdm_acc = %6.8f/%6.7f | senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f " % (
                                                                                                step, t_steps, loss_tmp[0].mean(),        
                                                                                            loss_tmp[1].mean(), acc_tmp[0].mean(),
                                                                                            loss_tmp[2].mean(), acc_tmp[1].mean(),
                                                                                            loss_tmp[3].mean(), acc_tmp[2].mean()
            )
            )
        
        if step % 10 == 9:
            sum_loss = sum_loss / 10
            sum_acc = sum_acc / 10
            print("MTL_Loss:%6.8f, rdm_loss/rdm_acc = %6.8f/%6.7f | senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f " % (
                                                                                            sum_loss[0],        
                                                                                            sum_loss[1], sum_acc[0],
                                                                                            sum_loss[2], sum_acc[1],
                                                                                            sum_loss[3], sum_acc[2]
            )
            )
            if step%100 == 99:
                rdm_save_as = '/home/hadoop/ERD/%s/rdmModel_epoch%03d.pkl'% (log_dir, step/100, sum_acc)
                torch.save(
                    {
                        "bert":bert.state_dict(),
                        "transformer":transformer.state_dict(),
                        "task_embedding":task_embedding.state_dict(),
                        "senti_classifier": senti_classifier.state_dict(),
                        "subj_classifier": subj_classifier.state_dict(),
                        "rmdModel":rdm_model.state_dict(),
                        "rdm_classifier": rdm_classifierdm.state_dict()
                    },
                    rdm_save_as
                )
#                 rdm_model, bert, sentiModel, rdm_classifier
            sum_acc = 0.0
            sum_loss = 0.0

    print(get_curtime() + " Train df Model End.")
    logger.info(get_curtime() + " Train df Model End.")
    return ret_acc

### 主函数部分

#### 模型部分

In [11]:
import transformer_utils

In [12]:
importlib.reload(transformer_utils)

<module 'transformer_utils' from '/home/hadoop/ERD/transformer_utils.py'>

In [13]:
tt = BertTokenizer.from_pretrained("./bertModel/")
bert = BertModel.from_pretrained("./bertModel/").cuda()

```
bb.save_pretrained('./bertModel/')  # save
tt.save_pretrained('./bertModel/')  # save
```

In [14]:
cm_model = CM_Model(300, 256, 2).cuda()
rdm_model = RDM_Model(768, 300, 256, 0.2).cuda()
rdm_classifier = nn.Linear(256, 2).cuda()

/home/hadoop/.conda/envs/torch_B/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [15]:
# encoder_layer = nn.TransformerEncoderLayer(768, 8)
# transformer_encoder = nn.TransformerEncoder(encoder_layer, 1)
trans_conf = adict({
  "attention_probs_dropout_prob": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 1,
  "num_labels": 2,
  "output_attentions": False,
  "output_hidden_states": False,
  "torchscript": False
})
BertEncoder = transformer_utils.BertEncoder
transformer = BertEncoder(trans_conf).cuda()

In [16]:
task_embedding = nn.Embedding(3, 768).cuda()

In [17]:
subj_pooler = nn.Sequential(nn.Linear(bert.config.hidden_size, bert.config.hidden_size), nn.Tanh()).cuda()
subj_cls = nn.Linear(768, 2).cuda()

In [18]:
senti_pooler = nn.Sequential(nn.Linear(bert.config.hidden_size, bert.config.hidden_size), nn.Tanh()).cuda()
senti_cls = nn.Linear(768, 2).cuda()

#### 各个任务的数据

In [11]:
subj_file = "./rotten_imdb/subj.data"
obj_file = "./rotten_imdb/obj.data"
tr, dev, te = load_data(subj_file, obj_file)

subj_train_reader = SubjObjReader(tr, 20, tt)
subj_valid_reader = SubjObjReader(dev, 20, tt)
subj_test_reader =  SubjObjReader(te, 20, tt)

In [24]:
train_file = "./SST-2/train.tsv"
dev_file = "./SST-2/dev.tsv"
train_data = SubjObjLoader.load_SST_from_file(train_file)
dev_data = SubjObjLoader.load_SST_from_file(dev_file)

In [25]:
sst_train_reader = SubjObjLoader.SubjObjReader(train_data[:-1000], 20, tt)

KeyboardInterrupt: 

In [26]:
sst_valid_reader = SubjObjLoader.SubjObjReader(dev_data, 20, tt)

In [ ]:
sst_test_reader = SubjObjLoader.SubjObjReader(train_data[-1000:], 20, tt)

In [16]:
train_file = "./trainingandtestdata/training.1600000.processed.noemoticon.csv"
test_file = "./trainingandtestdata/testdata.manual.2009.06.14.csv"
train_set, test_set = tsentiLoader.load_data(train_file, test_file)

./trainingandtestdata/training.1600000.processed.noemoticon.csv
./trainingandtestdata/testdata.manual.2009.06.14.csv


In [17]:
senti_train_reader = tsentiLoader.tSentiReader(train_set[:10000], 20, tt)
senti_train_reader.label = np.delete(senti_train_reader.label, 1, axis=2)
senti_valid_reader = tsentiLoader.tSentiReader(train_set[10000:10500], 20, tt)
senti_valid_reader.label = np.delete(senti_valid_reader.label, 1, axis=2)
# senti_test_reader =  tsentiLoader.tSentiReader(test_set, 20, tt)

In [50]:
load_data_fast()

max_sent: 187 ,  max_seq_len: 101
5802 data loaded


In [51]:
del data
del data_ID
del data_len
del data_y
from BertRDMLoader import data
from BertRDMLoader import data_len
from BertRDMLoader import data_ID
from BertRDMLoader import data_y

In [52]:
len(data_ID)

4084

In [32]:
def get_new_len(tokenizer, bert, rdm_model, cm_model, FLAGS, cuda):
    batch_size = 20
    new_len = []
    if len(data_ID) % batch_size == 0: # the total number of events
            flags = int(len(data_ID) / FLAGS.batch_size)
    else:
        flags = int(len(data_ID) / FLAGS.batch_size) + 1
    for i in range(flags):
        with torch.no_grad():
            x, x_len, y = get_df_batch(i, batch_size, tokenizer=tokenizer)
            sent_tensors, attn_mask, seq_len = rdm_data2bert_tensors(x, cuda)
            bert_outs = bert(sent_tensors, attention_mask=attn_mask)
            pooled_sents = [bert_outs[1][sum(seq_len[:idx]):sum(seq_len[:idx])+seq_len[idx]] for idx, s_len in enumerate(seq_len)]
            x_emb = rnn_utils.pad_sequence(pooled_sents, batch_first=True).unsqueeze(-2)
            batchsize, max_seq_len, max_sent_len, emb_dim = x_emb.shape
            rdm_hiddens = rdm_model(x_emb)
            batchsize, max_seq_len1, _ = rdm_hiddens.shape
#             rdm_outs = torch.cat(
#                 [ rdm_hiddens[i][x_len[i]-1] for i in range(batchsize)] 
#                 # a list of tensor, where the ndim of tensor is 1 and the shape of tensor is [hidden_size]
#             ).reshape(
#                 [-1, rdm_model.hidden_dim]
#             )
            stopScores = cm_model.Classifier(
                    nn.functional.relu(
                        cm_model.DenseLayer(
                            rdm_hiddens.reshape([-1, rdm_model.hidden_dim])
                    )
                )
            ).reshape(
                [batchsize, -1, 2]
            )
            _, max_seq_len2, _ = stopScores.shape
            assert max_seq_len1 == max_seq_len2
            isStop = stopScores.argmax(axis=-1).cpu().numpy()
            new_len.extend([iS.argmax()+1 if iS.argmax() <= x_len[i] and iS.max() ==1 else x_len[i] for i, iS in enumerate(isStop)])
        print("batch: ", i)
    return new_len

```python
def get_new_len(sess, mm):
    new_x_len = np.zeros([len(data_ID)], dtype=np.int32)
    for i in range(len(data_ID)):
        init_state = np.zeros([1, FLAGS.hidden_dim], dtype=np.float32)
        e_state = sess.run(mm.df_state, feed_dict={mm.init_states: init_state})
        for j in range(data_len[i]):
            t_words = data[data_ID[i]]['text'][j]
            e_x = np.zeros([1, FLAGS.max_sent_len, FLAGS.embedding_dim], dtype=np.float32)
            for k in range(len(t_words)):
                m_word = t_words[k]
                try:
                    e_x[0][k] = word2vec[m_word]
                except:
                    miss_word = 1
            # if len(t_words) != 0:
            #     e_x[0][:len(t_words)] = c2vec.vectorize_words(t_words)
            batch_dic = {mm.rl_state: e_state, mm.rl_input: e_x, mm.dropout_keep_prob: 1.0}
            e_isStop, mNewState = sess.run([mm.isStop, mm.rl_new_state], batch_dic)
            e_state = mNewState

            if e_isStop == 1:
                new_x_len[i] = j+1
                break
        if new_x_len[i] == 0 or new_x_len[i] > data_len[i]:
            new_x_len[i] = data_len[i]
    return new_x_len
```

In [ ]:
def get_new_len(tokenizer, bert, rdm_model, cm_model, FLAGS, cuda):
    batch_size = 20
    new_len = np.zeros([len(data_ID)], dtype=np.int32)
    for i in range(len(data_ID)):
        init_state = torch.zeros([1, FLAGS.hidden_dim], dtype=torch.float32)
        for j in range(data_len[i]):
            with torch.no_grad():
                input_ids = tokenizer.encode(data[data_ID[i]]['text'][j], add_special_token=True)
                _, sent_emb = bert(torch.tensor([input_ids]))
                
            e_x = np.zeros([1, FLAGS.max_sent_len, FLAGS.embedding_dim], dtype=np.float32)
            for k in range(len(t_words)):
                m_word = t_words[k]
                try:
                    e_x[0][k] = word2vec[m_word]
                except:
                    miss_word = 1
            # if len(t_words) != 0:
            #     e_x[0][:len(t_words)] = c2vec.vectorize_words(t_words)
            batch_dic = {mm.rl_state: e_state, mm.rl_input: e_x, mm.dropout_keep_prob: 1.0}
            e_isStop, mNewState = sess.run([mm.isStop, mm.rl_new_state], batch_dic)
            e_state = mNewState

            if e_isStop == 1:
                new_x_len[i] = j+1
                break
        if new_x_len[i] == 0 or new_x_len[i] > data_len[i]:
            new_x_len[i] = data_len[i]
    return new_len

In [18]:
tokenizer = tt
cuda = True

In [28]:
batch_size = 20
new_len = []
if len(data_ID) % batch_size == 0: # the total number of events
        flags = int(len(data_ID) / FLAGS.batch_size)
else:
    flags = int(len(data_ID) / FLAGS.batch_size) + 1
for i in range(1):
    with torch.no_grad():
        x, x_len, y = get_df_batch(i, batch_size, tokenizer=tokenizer)
        sent_tensors, attn_mask, seq_len = rdm_data2bert_tensors(x, cuda)
        bert_outs = bert(sent_tensors, attention_mask=attn_mask)
        pooled_sents = [bert_outs[1][sum(seq_len[:idx]):sum(seq_len[:idx])+seq_len[idx]] for idx, s_len in enumerate(seq_len)]
        x_emb = rnn_utils.pad_sequence(pooled_sents, batch_first=True).unsqueeze(-2)
        batchsize, max_seq_len, max_sent_len, emb_dim = x_emb.shape
        rdm_hiddens = rdm_model(x_emb)
        batchsize, _, _ = rdm_hiddens.shape
        rdm_outs = torch.cat(
            [ rdm_hiddens[i][x_len[i]-1] for i in range(batchsize)] 
            # a list of tensor, where the ndim of tensor is 1 and the shape of tensor is [hidden_size]
        ).reshape(
            [-1, rdm_model.hidden_dim]
        )
        stopScores = cm_model.Classifier(
                nn.functional.relu(
                    cm_model.DenseLayer(
                        rdm_hiddens.reshape([-1, rdm_model.hidden_dim])
                )
            )
        ).reshape(
            [batchsize, -1, 2]
        )
        isStop = stopScores.argmax(axis=-1).cpu().numpy()
        new_len.extend([iS.argmax() if iS.argmax() <= x_len[i] and iS.max() ==1 else x_len[i] for i, iS in enumerate(isStop)])
    print("batch: ", i)

batch:  0


这里的逻辑是，一次性将所有的rdm_hiddens都给算出来，然后再使用CM 模型去判定，再挑选最早输出终止的那个位置作为终止位置。如此，便能得到序列在CM 模型判定下的长度。　

而原来的逻辑是，一个一个进行判断，这两者的逻辑应该是等价的。

可是为什么那样它能得到更长的序列长度呢, 是因为这个模型的训练次数不够，致使CM模型一般化吗？

In [31]:
s = max([])

ValueError: max() arg is an empty sequence

In [36]:
device_ids = [0]
bert = nn.DataParallel(bert, device_ids=device_ids)
device_name = "cuda:%d"%device_ids[0]
device = torch.device(device_name)
bert.to(device)

DataParallel(
  (module): DataParallel(
    (module): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
            

In [39]:
pretrained_file = '../Downloads/erdModel_epoch000(1).pkl'
checkpoint = torch.load(pretrained_file)
bert.module.load_state_dict(checkpoint['bert'])
rdm_model.load_state_dict(checkpoint["rmdModel"])
cm_model.load_state_dict(checkpoint['cm_model'])
rdm_classifier.load_state_dict(checkpoint["rdm_classifier"])

<All keys matched successfully>

In [57]:
reward_counter = 0

In [58]:
def get_reward(isStop, ss, pys, ids, seq_ids):
    global reward_counter
    reward = torch.zeros([len(isStop)], dtype=torch.float32)
    Q_Val = torch.zeros([len(isStop)], dtype= torch.float32)
    for i in range(len(isStop)):
        if isStop[i] == 1:
            if pys[ids[i]][seq_ids[i]-1].argmax() == np.argmax(data_y[ids[i]]):
                # 在这个位置停止之后，如果使用这个位置的输出来判定，确实是与标签一致，那么就奖励
                reward_counter += 1 # more number of correct prediction, more rewards
                r = 1 + FLAGS.reward_rate * math.log(reward_counter)
                reward[i] = r   
            else:
                reward[i] = -100
            Q_Val[i] = reward[i]
        else:
            reward[i] = -0.01 
            Q_Val[i] = reward[i] + 0.99 * max(ss[i])
    return reward, Q_Val

In [40]:
new_len = get_new_len(tt, bert, rdm_model, cm_model, FLAGS, cuda=True)

batch:  0
batch:  1
batch:  2
batch:  3
batch:  4
batch:  5
batch:  6
batch:  7
batch:  8
batch:  9
batch:  10
batch:  11
batch:  12
batch:  13
batch:  14
batch:  15
batch:  16
batch:  17
batch:  18
batch:  19
batch:  20
batch:  21
batch:  22
batch:  23
batch:  24
batch:  25
batch:  26
batch:  27
batch:  28
batch:  29
batch:  30
batch:  31
batch:  32
batch:  33
batch:  34
batch:  35
batch:  36
batch:  37
batch:  38
batch:  39
batch:  40
batch:  41
batch:  42
batch:  43
batch:  44
batch:  45
batch:  46
batch:  47
batch:  48
batch:  49
batch:  50
batch:  51
batch:  52
batch:  53
batch:  54
batch:  55
batch:  56
batch:  57
batch:  58
batch:  59
batch:  60
batch:  61
batch:  62
batch:  63
batch:  64
batch:  65
batch:  66
batch:  67
batch:  68
batch:  69
batch:  70
batch:  71
batch:  72
batch:  73
batch:  74
batch:  75
batch:  76
batch:  77
batch:  78
batch:  79
batch:  80
batch:  81
batch:  82
batch:  83
batch:  84
batch:  85
batch:  86
batch:  87
batch:  88
batch:  89
batch:  90
batch:  9

In [46]:
len(data_len), len(new_len)

(3884, 3900)

In [56]:
(1 - (np.array(data_len[:3884]) - np.array(new_len[:3884])) /np.array(data_len[:3884])).tolist()

[0.05555555555555558,
 0.0714285714285714,
 0.06666666666666665,
 1.0,
 0.05555555555555558,
 0.0625,
 1.0,
 0.010638297872340385,
 0.125,
 0.07692307692307687,
 0.125,
 1.0,
 1.0,
 0.038461538461538436,
 1.0,
 0.16666666666666663,
 0.09090909090909094,
 0.040000000000000036,
 1.0,
 1.0,
 0.052631578947368474,
 0.06666666666666665,
 1.0,
 0.01666666666666672,
 0.16666666666666663,
 0.05882352941176472,
 1.0,
 0.04166666666666663,
 0.07692307692307687,
 0.00990099009900991,
 0.02777777777777779,
 0.03125,
 0.16666666666666663,
 0.02631578947368418,
 1.0,
 0.05555555555555558,
 1.0,
 1.0,
 0.09090909090909094,
 1.0,
 1.0,
 0.050000000000000044,
 0.00990099009900991,
 1.0,
 0.04166666666666663,
 1.0,
 1.0,
 0.04761904761904767,
 1.0,
 1.0,
 1.0,
 0.01754385964912286,
 0.04166666666666663,
 0.16666666666666663,
 0.04347826086956519,
 0.04761904761904767,
 0.050000000000000044,
 1.0,
 0.06666666666666665,
 1.0,
 1.0,
 1.0,
 0.05555555555555558,
 1.0,
 0.125,
 0.052631578947368474,
 0.0625,


[1,
 1,
 1,
 15,
 1,
 1,
 14,
 1,
 1,
 1,
 1,
 8,
 101,
 1,
 25,
 1,
 1,
 1,
 12,
 28,
 1,
 1,
 23,
 1,
 1,
 1,
 16,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 30,
 1,
 9,
 15,
 1,
 10,
 6,
 1,
 1,
 9,
 1,
 20,
 12,
 1,
 18,
 6,
 9,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 14,
 18,
 18,
 1,
 9,
 1,
 1,
 1,
 1,
 1,
 28,
 13,
 1,
 10,
 1,
 34,
 1,
 1,
 40,
 1,
 1,
 24,
 1,
 10,
 1,
 1,
 1,
 14,
 20,
 18,
 1,
 1,
 18,
 22,
 1,
 1,
 1,
 16,
 1,
 1,
 1,
 1,
 26,
 24,
 1,
 10,
 7,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 45,
 12,
 1,
 1,
 8,
 1,
 1,
 13,
 7,
 1,
 55,
 1,
 1,
 1,
 51,
 16,
 1,
 1,
 1,
 6,
 19,
 1,
 13,
 1,
 1,
 1,
 1,
 1,
 1,
 21,
 1,
 1,
 1,
 1,
 10,
 1,
 1,
 1,
 20,
 10,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 20,
 1,
 1,
 1,
 12,
 17,
 1,
 1,
 1,
 1,
 1,
 9,
 16,
 1,
 67,
 23,
 1,
 21,
 1,
 1,
 1,
 9,
 1,
 1,
 1,
 1,
 21,
 1,
 1,
 11,
 1,
 1,
 12,
 1,
 1,
 1,
 1,
 1,
 17,
 1,
 1,
 18,
 1,
 1,
 1,
 1,
 1,
 1,
 20,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 19,
 17,
 20,
 22,
 1,
 1,
 1,
 1,
 1,
 8,
 7,
 1,
 27,
 19,
 1

In [9]:
sum(data_len)

79572

In [17]:
cuda = True

In [16]:
x, x_len, y = get_df_batch(0, 5, [], tokenizer=tt)

In [18]:
with torch.no_grad():
    sent_tensors, attn_mask, seq_len = rdm_data2bert_tensors(x, cuda)
    bert_outs = bert(sent_tensors, attention_mask=attn_mask)
    pooled_sents = [bert_outs[1][sum(seq_len[:idx]):sum(seq_len[:idx])+seq_len[idx]] for idx, s_len in enumerate(seq_len)]
    data_tensors = rnn_utils.pad_sequence(pooled_sents, batch_first=True)
    seq_mask = torch.ones([len(seq_len), max(seq_len)]).cuda()
    for i in range(len(seq_len)):
        seq_mask[i][seq_len[i]:].fill_(0)
    rdm_transformer(data_tensors, attention_mask = sent_mask)

### 调试各个任务的部分

#### 调试subjective的任务

In [23]:
 subj_cls_train(subj_train_reader, subj_valid_reader, 
                   bert, task_embedding, subj_pooler, subj_cls, 
                   2, cuda=True, log_dir="SubjRDM"
                  )

step:0 | loss/acc = 0.016/1.000
step:1 | loss/acc = 7.076/0.450
step:2 | loss/acc = 0.091/0.950
step:3 | loss/acc = 1.131/0.700
step:4 | loss/acc = 0.279/0.900
step:5 | loss/acc = 0.059/1.000
step:6 | loss/acc = 0.028/1.000
step:7 | loss/acc = 0.016/1.000
step:8 | loss/acc = 0.011/1.000
step:9 | loss/acc = 0.007/1.000
subjective task:      9: [    0/    2], subj_loss/subj_acc = 0.87133088/0.9000000 
step:10 | loss/acc = 0.005/1.000
step:11 | loss/acc = 0.045/0.950
step:12 | loss/acc = 0.182/0.950
step:13 | loss/acc = 0.229/0.900
step:14 | loss/acc = 0.141/0.900
step:15 | loss/acc = 0.062/1.000
step:16 | loss/acc = 0.140/0.900
step:17 | loss/acc = 0.010/1.000
step:18 | loss/acc = 0.009/1.000
step:19 | loss/acc = 0.017/1.000
subjective task:     19: [    0/    2], subj_loss/subj_acc = 0.08406559/0.9600000 
step:20 | loss/acc = 0.014/1.000
step:21 | loss/acc = 0.004/1.000
step:22 | loss/acc = 0.003/1.000
step:23 | loss/acc = 0.011/1.000
step:24 | loss/acc = 0.006/1.000
step:25 | loss/acc 

step:199 | loss/acc = 0.008/1.000
subjective task:    199: [    0/    2], subj_loss/subj_acc = 0.05264260/0.9850000 
step:200 | loss/acc = 0.078/0.950
step:201 | loss/acc = 0.004/1.000
step:202 | loss/acc = 0.033/1.000
step:203 | loss/acc = 0.010/1.000
step:204 | loss/acc = 0.040/0.950
step:205 | loss/acc = 0.087/0.950
step:206 | loss/acc = 0.012/1.000
step:207 | loss/acc = 0.017/1.000
step:208 | loss/acc = 0.064/0.950
step:209 | loss/acc = 0.002/1.000
subjective task:    209: [    0/    2], subj_loss/subj_acc = 0.03492010/0.9800000 
step:210 | loss/acc = 0.108/0.950
step:211 | loss/acc = 0.002/1.000
step:212 | loss/acc = 0.002/1.000
step:213 | loss/acc = 0.060/0.950
step:214 | loss/acc = 0.008/1.000
step:215 | loss/acc = 0.002/1.000
step:216 | loss/acc = 0.010/1.000
step:217 | loss/acc = 0.006/1.000
step:218 | loss/acc = 0.035/1.000
step:219 | loss/acc = 0.021/1.000
subjective task:    219: [    0/    2], subj_loss/subj_acc = 0.02536672/0.9900000 
step:220 | loss/acc = 0.002/1.000
ste

step:392 | loss/acc = 0.174/0.950
step:393 | loss/acc = 0.088/0.950
step:394 | loss/acc = 0.014/1.000
step:395 | loss/acc = 0.025/1.000
step:396 | loss/acc = 0.031/1.000
step:397 | loss/acc = 0.237/0.950
step:398 | loss/acc = 0.018/1.000
step:399 | loss/acc = 0.048/1.000
subjective task:    399: [    0/    2], subj_loss/subj_acc = 0.07407559/0.9800000 
step:400 | loss/acc = 0.024/1.000
step:401 | loss/acc = 0.041/1.000
step:402 | loss/acc = 0.008/1.000
step:403 | loss/acc = 0.048/1.000
step:404 | loss/acc = 0.014/1.000
step:405 | loss/acc = 0.009/1.000
step:406 | loss/acc = 0.018/1.000
step:407 | loss/acc = 0.004/1.000
step:408 | loss/acc = 0.024/1.000
step:409 | loss/acc = 0.007/1.000
subjective task:    409: [    0/    2], subj_loss/subj_acc = 0.01978597/1.0000000 
step:410 | loss/acc = 0.009/1.000
step:411 | loss/acc = 0.051/1.000
step:412 | loss/acc = 0.143/0.950
step:413 | loss/acc = 0.004/1.000
step:414 | loss/acc = 0.066/0.950
step:415 | loss/acc = 0.039/1.000
step:416 | loss/ac

NameError: name 'transformer' is not defined

In [ ]:
 subj_cls_train(subj_train_reader, subj_valid_reader, 
                   bert, transformer, task_embedding, subj_classifier, 
                   2, cuda=True, log_dir="SubjRDM"
                  )

In [19]:
joint_save_as = './SubjRDM/subj_best_Model.pkl'
checkpoint = torch.load(joint_save_as)
bert.load_state_dict(checkpoint['bert'])
transformer.load_state_dict(checkpoint['transformer'])
task_embedding.load_state_dict(checkpoint['task_embedding'])
subj_cls.load_state_dict(checkpoint['subj_classifier'])

<All keys matched successfully>

> __test code__
``` python
for xsj, ysj, lsj in subj_train_reader.iter():
    loss = subj_loss(  xsj, ysj, lsj,
                bb, transformer_encoder, task_embedding(torch.tensor([1])),
                subj_cls, subj_loss_fn
               )
    break
```
> __train stage__
```python
subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_train_reader.label
            ),
            dtype = torch.float32
    ).cuda()
subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights)
subj_cls_train(subj_train_reader, bert, transformer, task_embedding, subj_cls, 1, subj_loss_fn, cuda=True)
```
_subj task 的训练，在当前的参数配置下，一个ｅｐｏｃｈ可以调到很好的状态_

In [21]:
cuda = True
subj_task_id = torch.tensor([1]).cuda() if cuda else torch.tensor([1])

In [23]:
with torch.no_grad():
    bs_cnt, bs, l_cnt = subj_valid_reader.label.shape
    preds = []
    losses = np.zeros(bs_cnt)
    it = 0
    for xsj, ysj, lsj in subj_valid_reader.iter():
        sent_tensors, sent_mask = senti_data2bert_tensors(xsj, cuda)
        xsj_embs, _ = bert(sent_tensors, attention_mask = sent_mask)
        tensors = xsj_embs + task_embedding(subj_task_id)
        subj_feature = transformer(tensors, attention_mask = sent_mask)
        cls_feature = subj_feature[0][:, 0]
        subj_scores = subj_cls(cls_feature)
        y_label = torch.tensor(ysj.argmax(axis=1)).cuda() if cuda else torch.tensor(ysj.argmax(axis=1))
        sj_acc = accuracy_score(ysj.argmax(axis=1), subj_scores.cpu().argmax(axis=1))
        preds.append(subj_scores)
        torch.cuda.empty_cache()
        it += 1
    val_preds = torch.cat(preds).cpu()
    val_acc = accuracy_score(subj_valid_reader.label.reshape(bs_cnt*bs, l_cnt).argmax(axis=1), val_preds.argmax(axis=1))
    print("valid loss/acc: %.6f/%.6f:"%(0, val_acc))

valid loss/acc: 0.000000/0.940000:


#### 调试sentiment 的任务

In [18]:
cuda = True
senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()

In [29]:
joint_save_as = './SentiRDM/senti_best_Model.pkl'
checkpoint = torch.load(joint_save_as)
senti_cls.load_state_dict(checkpoint['senti_classifier'])
bert.load_state_dict(checkpoint['bert'])
transformer.load_state_dict(checkpoint['transformer'])
task_embedding.load_state_dict(checkpoint['task_embedding'])

<All keys matched successfully>

In [121]:
seq_mask

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
       device='cuda:0')

In [116]:
seq_len

[18, 14, 15, 15, 18]

In [122]:
with torch.no_grad():
    rst = transformer(data_tensors, attention_mask = seq_mask)

In [115]:
rst[0].shape

torch.Size([5, 18, 768])

In [123]:
rst[0][1][0]

tensor([-0.7457, -0.2687, -0.6856,  1.3910, -0.0797,  0.0575,  1.3677, -0.2973,
         0.2637, -1.9420,  0.0614,  0.7212,  1.6432, -0.2240,  1.5888, -0.0353,
         0.0626, -0.1229,  0.2974,  0.9160,  1.0717,  2.1583,  0.4307,  0.5733,
         0.0666,  1.4145, -0.6543,  1.0156,  1.9368,  0.7030, -0.7259, -0.3737,
        -2.2893, -0.1830, -2.0724, -2.1778,  0.4458, -1.7380,  0.4244,  0.4769,
        -1.3549,  0.3928,  1.7463, -0.0545,  0.0307, -0.7469, -1.7027,  0.1404,
        -1.2502,  0.9609,  1.1209,  0.4120,  0.7475,  0.7332,  1.0633,  0.5057,
        -1.1561,  1.0175, -0.5818, -0.4796, -0.6358,  0.4046, -1.9045, -1.6574,
         0.4951,  0.9133, -0.1089,  0.3090,  0.0999,  0.1044,  1.7216,  0.0169,
         0.5352, -1.0694,  0.5556,  0.1750, -0.9352,  1.8241,  0.1404, -2.2726,
         0.8597,  0.6773,  1.2350,  0.4602,  0.5165, -1.8105,  0.2405, -0.6124,
        -1.5991,  0.8526,  1.0232,  0.1471,  0.4354,  1.2182, -0.4516,  0.0333,
        -0.5293, -0.7726, -1.0948,  0.34

In [30]:
with torch.no_grad():
    bs_cnt, bs, l_cnt = sst_valid_reader.label.shape
    preds = []
    losses = np.zeros(bs_cnt)
    it = 0
    for xst, yst, lst in sst_valid_reader.iter():
        sent_tensors, sent_mask = senti_data2bert_tensors(xst, cuda)
        xst_embs, _ = bert(sent_tensors, attention_mask = sent_mask)
        tensors = xst_embs + task_embedding(senti_task_id)
        senti_feature = transformer(tensors, attention_mask = sent_mask)
        cls_feature = senti_feature[0][:, 0]
        senti_scores = senti_cls(cls_feature)
        y_label = torch.tensor(yst.argmax(axis=1)).cuda() if cuda else torch.tensor(yst.argmax(axis=1))
        st_acc = accuracy_score(yst.argmax(axis=1), senti_scores.cpu().argmax(axis=1))
        preds.append(senti_scores)
        torch.cuda.empty_cache()
    val_preds = torch.cat(preds).cpu()
    val_acc = accuracy_score(sst_valid_reader.label.reshape(bs_cnt*bs, l_cnt).argmax(axis=1), val_preds.argmax(axis=1))
    print("valid loss/acc: %.6f/%.6f:"%(0, val_acc))

valid loss/acc: 0.000000/0.919767:


In [25]:
senti_cls_train(senti_train_reader, senti_valid_reader,
                    bert, task_embedding, senti_pooler, senti_cls,
                    10, cuda=True, log_dir="SentiRDM"
                   )

step:0 | loss/acc = 0.684/0.600
step:1 | loss/acc = 0.795/0.450
step:2 | loss/acc = 0.640/0.700
step:3 | loss/acc = 0.732/0.400
step:4 | loss/acc = 0.683/0.600
step:5 | loss/acc = 0.697/0.400
step:6 | loss/acc = 0.695/0.450
step:7 | loss/acc = 0.673/0.800
step:8 | loss/acc = 0.677/0.650
step:9 | loss/acc = 0.703/0.450
sentiment task:      9: [    0/   10], senti_loss/senti_acc = 0.69775417/0.5500000 
step:10 | loss/acc = 0.650/0.600
step:11 | loss/acc = 0.723/0.450
step:12 | loss/acc = 0.675/0.650
step:13 | loss/acc = 0.666/0.500
step:14 | loss/acc = 0.631/0.700
step:15 | loss/acc = 0.632/0.700
step:16 | loss/acc = 0.652/0.500
step:17 | loss/acc = 0.661/0.550
step:18 | loss/acc = 0.652/0.550
step:19 | loss/acc = 0.558/0.700
sentiment task:     19: [    0/   10], senti_loss/senti_acc = 0.65003407/0.5900000 
step:20 | loss/acc = 0.644/0.600
step:21 | loss/acc = 0.603/0.650
step:22 | loss/acc = 0.535/0.750
step:23 | loss/acc = 0.585/0.900
step:24 | loss/acc = 0.588/0.750
step:25 | loss/ac

step:199 | loss/acc = 0.603/0.700
sentiment task:    199: [    0/   10], senti_loss/senti_acc = 0.48256984/0.7650000 
step:200 | loss/acc = 0.462/0.750
step:201 | loss/acc = 0.340/0.850
step:202 | loss/acc = 0.313/0.850
step:203 | loss/acc = 0.572/0.750
step:204 | loss/acc = 0.349/0.850
step:205 | loss/acc = 0.363/0.900
step:206 | loss/acc = 0.769/0.600
step:207 | loss/acc = 0.382/0.850
step:208 | loss/acc = 0.346/0.800
step:209 | loss/acc = 0.612/0.750
sentiment task:    209: [    0/   10], senti_loss/senti_acc = 0.45076599/0.7950000 
step:210 | loss/acc = 0.363/0.850
step:211 | loss/acc = 0.389/0.850
step:212 | loss/acc = 0.166/1.000
step:213 | loss/acc = 0.690/0.650
step:214 | loss/acc = 0.534/0.750
step:215 | loss/acc = 0.376/0.750
step:216 | loss/acc = 0.578/0.800
step:217 | loss/acc = 0.310/0.900
step:218 | loss/acc = 0.459/0.800
step:219 | loss/acc = 0.220/0.950
sentiment task:    219: [    0/   10], senti_loss/senti_acc = 0.40851160/0.8300000 
step:220 | loss/acc = 0.461/0.750


step:392 | loss/acc = 0.520/0.800
step:393 | loss/acc = 0.414/0.800
step:394 | loss/acc = 0.325/0.850
step:395 | loss/acc = 0.504/0.750
step:396 | loss/acc = 0.340/0.850
step:397 | loss/acc = 0.589/0.800
step:398 | loss/acc = 0.444/0.800
step:399 | loss/acc = 0.242/1.000
sentiment task:    399: [    0/   10], senti_loss/senti_acc = 0.41332280/0.8300000 
step:400 | loss/acc = 0.394/0.800
step:401 | loss/acc = 0.340/0.850
step:402 | loss/acc = 0.560/0.750
step:403 | loss/acc = 0.271/0.900
step:404 | loss/acc = 0.367/0.800
step:405 | loss/acc = 0.426/0.750
step:406 | loss/acc = 0.342/0.900
step:407 | loss/acc = 0.431/0.800
step:408 | loss/acc = 0.452/0.700
step:409 | loss/acc = 0.526/0.800
sentiment task:    409: [    0/   10], senti_loss/senti_acc = 0.41102981/0.8050000 
step:410 | loss/acc = 0.387/0.850
step:411 | loss/acc = 0.413/0.900
step:412 | loss/acc = 0.376/0.800
step:413 | loss/acc = 0.541/0.750
step:414 | loss/acc = 0.600/0.750
step:415 | loss/acc = 0.459/0.750
step:416 | loss/

step:585 | loss/acc = 0.277/0.900
step:586 | loss/acc = 0.351/0.850
step:587 | loss/acc = 0.085/1.000
step:588 | loss/acc = 0.374/0.850
step:589 | loss/acc = 0.239/0.900
sentiment task:    589: [    1/   10], senti_loss/senti_acc = 0.11504819/0.8950000 
step:590 | loss/acc = 0.094/1.000
step:591 | loss/acc = 0.252/0.900
step:592 | loss/acc = 0.342/0.950
step:593 | loss/acc = 0.135/0.950
step:594 | loss/acc = 0.257/0.950
step:595 | loss/acc = 0.220/0.900
step:596 | loss/acc = 0.211/0.850
step:597 | loss/acc = 0.130/0.900
step:598 | loss/acc = 0.167/0.950
step:599 | loss/acc = 0.200/0.850
sentiment task:    599: [    1/   10], senti_loss/senti_acc = 0.08032541/0.9200000 
step:600 | loss/acc = 0.222/0.900
step:601 | loss/acc = 0.175/0.900
step:602 | loss/acc = 0.428/0.900
step:603 | loss/acc = 0.044/1.000
step:604 | loss/acc = 0.297/0.900
step:605 | loss/acc = 0.107/0.950
step:606 | loss/acc = 0.126/0.950
step:607 | loss/acc = 0.254/0.900
step:608 | loss/acc = 0.222/0.850
step:609 | loss/

step:780 | loss/acc = 0.116/0.950
step:781 | loss/acc = 0.355/0.900
step:782 | loss/acc = 0.099/1.000
step:783 | loss/acc = 0.102/0.950
step:784 | loss/acc = 0.360/0.900
step:785 | loss/acc = 0.206/0.850
step:786 | loss/acc = 0.364/0.800
step:787 | loss/acc = 0.152/0.950
step:788 | loss/acc = 0.296/0.900
step:789 | loss/acc = 0.161/0.950
sentiment task:    789: [    1/   10], senti_loss/senti_acc = 0.08841923/0.9150000 
step:790 | loss/acc = 0.135/0.950
step:791 | loss/acc = 0.126/0.900
step:792 | loss/acc = 0.139/1.000
step:793 | loss/acc = 0.212/0.850
step:794 | loss/acc = 0.185/0.950
step:795 | loss/acc = 0.173/0.950
step:796 | loss/acc = 0.131/0.950
step:797 | loss/acc = 0.219/0.900
step:798 | loss/acc = 0.448/0.800
step:799 | loss/acc = 0.158/0.900
sentiment task:    799: [    1/   10], senti_loss/senti_acc = 0.07704686/0.9150000 
step:800 | loss/acc = 0.148/0.950
step:801 | loss/acc = 0.277/0.850
step:802 | loss/acc = 0.353/0.800
step:803 | loss/acc = 0.379/0.800
step:804 | loss/

...

step:4200 | loss/acc = 0.000/1.000
step:4201 | loss/acc = 0.033/1.000
step:4202 | loss/acc = 0.000/1.000
step:4203 | loss/acc = 0.006/1.000
step:4204 | loss/acc = 0.000/1.000
step:4205 | loss/acc = 0.001/1.000
step:4206 | loss/acc = 0.019/1.000
step:4207 | loss/acc = 0.031/1.000
step:4208 | loss/acc = 0.078/0.950
step:4209 | loss/acc = 0.448/0.950
sentiment task:   4209: [    8/   10], senti_loss/senti_acc = 0.02468234/0.9900000 
step:4210 | loss/acc = 0.000/1.000
step:4211 | loss/acc = 0.001/1.000
step:4212 | loss/acc = 0.016/1.000
step:4213 | loss/acc = 0.030/1.000
step:4214 | loss/acc = 0.001/1.000
step:4215 | loss/acc = 0.003/1.000
step:4216 | loss/acc = 0.056/0.950
step:4217 | loss/acc = 0.007/1.000
step:4218 | loss/acc = 0.001/1.000
step:4219 | loss/acc = 0.000/1.000
sentiment task:   4219: [    8/   10], senti_loss/senti_acc = 0.00453800/0.9950000 
step:4220 | loss/acc = 0.023/1.000
step:4221 | loss/acc = 0.001/1.000
step:4222 | loss/acc = 0.158/0.950
step:4223 | loss/acc = 0.00

step:4391 | loss/acc = 0.003/1.000
step:4392 | loss/acc = 0.051/0.950
step:4393 | loss/acc = 0.000/1.000
step:4394 | loss/acc = 0.000/1.000
step:4395 | loss/acc = 0.002/1.000
step:4396 | loss/acc = 0.001/1.000
step:4397 | loss/acc = 0.001/1.000
step:4398 | loss/acc = 0.044/0.950
step:4399 | loss/acc = 0.020/1.000
sentiment task:   4399: [    8/   10], senti_loss/senti_acc = 0.00484229/0.9900000 
step:4400 | loss/acc = 0.055/0.950
step:4401 | loss/acc = 0.060/0.950
step:4402 | loss/acc = 0.020/1.000
step:4403 | loss/acc = 0.023/1.000
step:4404 | loss/acc = 0.000/1.000
step:4405 | loss/acc = 0.000/1.000
step:4406 | loss/acc = 0.000/1.000
step:4407 | loss/acc = 0.003/1.000
step:4408 | loss/acc = 0.061/0.950
step:4409 | loss/acc = 0.001/1.000
sentiment task:   4409: [    8/   10], senti_loss/senti_acc = 0.00898894/0.9850000 
step:4410 | loss/acc = 0.020/1.000
step:4411 | loss/acc = 0.005/1.000
step:4412 | loss/acc = 0.018/1.000
step:4413 | loss/acc = 0.005/1.000
step:4414 | loss/acc = 0.07

step:4580 | loss/acc = 0.000/1.000
step:4581 | loss/acc = 0.048/0.950
step:4582 | loss/acc = 0.000/1.000
step:4583 | loss/acc = 0.000/1.000
step:4584 | loss/acc = 0.002/1.000
step:4585 | loss/acc = 0.029/1.000
step:4586 | loss/acc = 0.098/0.950
step:4587 | loss/acc = 0.000/1.000
step:4588 | loss/acc = 0.000/1.000
step:4589 | loss/acc = 0.000/1.000
sentiment task:   4589: [    9/   10], senti_loss/senti_acc = 0.00714105/0.9900000 
step:4590 | loss/acc = 0.000/1.000
step:4591 | loss/acc = 0.000/1.000
step:4592 | loss/acc = 0.000/1.000
step:4593 | loss/acc = 0.000/1.000
step:4594 | loss/acc = 0.001/1.000
step:4595 | loss/acc = 0.000/1.000
step:4596 | loss/acc = 0.000/1.000
step:4597 | loss/acc = 0.000/1.000
step:4598 | loss/acc = 0.000/1.000
step:4599 | loss/acc = 0.001/1.000
sentiment task:   4599: [    9/   10], senti_loss/senti_acc = 0.00016349/1.0000000 
step:4600 | loss/acc = 0.001/1.000
step:4601 | loss/acc = 0.065/0.950
step:4602 | loss/acc = 0.000/1.000
step:4603 | loss/acc = 0.00

step:4771 | loss/acc = 0.085/0.950
step:4772 | loss/acc = 0.007/1.000
step:4773 | loss/acc = 0.000/1.000
step:4774 | loss/acc = 0.048/0.950
step:4775 | loss/acc = 0.000/1.000
step:4776 | loss/acc = 0.110/0.900
step:4777 | loss/acc = 0.000/1.000
step:4778 | loss/acc = 0.000/1.000
step:4779 | loss/acc = 0.042/0.950
sentiment task:   4779: [    9/   10], senti_loss/senti_acc = 0.01181638/0.9750000 
step:4780 | loss/acc = 0.015/1.000
step:4781 | loss/acc = 0.000/1.000
step:4782 | loss/acc = 0.075/0.950
step:4783 | loss/acc = 0.000/1.000
step:4784 | loss/acc = 0.000/1.000
step:4785 | loss/acc = 0.070/0.950
step:4786 | loss/acc = 0.000/1.000
step:4787 | loss/acc = 0.000/1.000
step:4788 | loss/acc = 0.001/1.000
step:4789 | loss/acc = 0.031/1.000
sentiment task:   4789: [    9/   10], senti_loss/senti_acc = 0.00766467/0.9900000 
step:4790 | loss/acc = 0.000/1.000
step:4791 | loss/acc = 0.000/1.000
step:4792 | loss/acc = 0.000/1.000
step:4793 | loss/acc = 0.092/0.900
step:4794 | loss/acc = 0.00

step:4961 | loss/acc = 0.093/0.950
step:4962 | loss/acc = 0.003/1.000
step:4963 | loss/acc = 0.003/1.000
step:4964 | loss/acc = 0.000/1.000
step:4965 | loss/acc = 0.000/1.000
step:4966 | loss/acc = 0.000/1.000
step:4967 | loss/acc = 0.001/1.000
step:4968 | loss/acc = 0.003/1.000
step:4969 | loss/acc = 0.051/0.950
sentiment task:   4969: [    9/   10], senti_loss/senti_acc = 0.00621222/0.9900000 
step:4970 | loss/acc = 0.075/0.950
step:4971 | loss/acc = 0.002/1.000
step:4972 | loss/acc = 0.003/1.000
step:4973 | loss/acc = 0.024/1.000
step:4974 | loss/acc = 0.000/1.000
step:4975 | loss/acc = 0.001/1.000
step:4976 | loss/acc = 0.000/1.000
step:4977 | loss/acc = 0.132/0.900
step:4978 | loss/acc = 0.001/1.000
step:4979 | loss/acc = 0.073/0.950
sentiment task:   4979: [    9/   10], senti_loss/senti_acc = 0.01242043/0.9800000 
step:4980 | loss/acc = 0.001/1.000
step:4981 | loss/acc = 0.000/1.000
step:4982 | loss/acc = 0.011/1.000
step:4983 | loss/acc = 0.000/1.000
step:4984 | loss/acc = 0.00

In [ ]:
senti_cls_train(sst_train_reader, sst_valid_reader,
                    bert, transformer, task_embedding, senti_cls,
                    3, cuda=True, log_dir="SentiRDM"
                   )

step:0 | loss/acc = 0.299/0.950
step:1 | loss/acc = 0.423/0.750
step:2 | loss/acc = 0.238/0.900
step:3 | loss/acc = 0.055/1.000
step:4 | loss/acc = 0.517/0.850
step:5 | loss/acc = 0.529/0.850
step:6 | loss/acc = 0.126/0.950
step:7 | loss/acc = 0.456/0.850
step:8 | loss/acc = 0.312/0.850
step:9 | loss/acc = 0.266/0.850
sentiment task:      9: [    0/    3], senti_loss/senti_acc = 0.32211870/0.8800000 
step:10 | loss/acc = 0.240/0.900
step:11 | loss/acc = 0.193/0.950
step:12 | loss/acc = 0.267/0.850
step:13 | loss/acc = 0.261/0.900
step:14 | loss/acc = 0.634/0.800
step:15 | loss/acc = 0.220/0.950
step:16 | loss/acc = 0.171/0.900
step:17 | loss/acc = 0.118/0.950
step:18 | loss/acc = 0.282/0.900
step:19 | loss/acc = 0.179/0.950
sentiment task:     19: [    0/    3], senti_loss/senti_acc = 0.25647906/0.9050000 
step:20 | loss/acc = 0.197/0.950
step:21 | loss/acc = 0.394/0.900
step:22 | loss/acc = 0.179/0.900
step:23 | loss/acc = 0.309/0.950
step:24 | loss/acc = 0.424/0.900
step:25 | loss/ac

step:198 | loss/acc = 0.575/0.800
step:199 | loss/acc = 0.140/1.000
sentiment task:    199: [    0/    3], senti_loss/senti_acc = 0.28698388/0.9000000 
step:200 | loss/acc = 0.125/1.000
step:201 | loss/acc = 0.523/0.900
step:202 | loss/acc = 0.429/0.800
step:203 | loss/acc = 0.129/0.950
step:204 | loss/acc = 0.425/0.800
step:205 | loss/acc = 0.399/0.800
step:206 | loss/acc = 0.214/0.950
step:207 | loss/acc = 0.342/0.850
step:208 | loss/acc = 0.182/0.950
step:209 | loss/acc = 0.471/0.850
sentiment task:    209: [    0/    3], senti_loss/senti_acc = 0.32399431/0.8850000 
step:210 | loss/acc = 0.176/0.950
step:211 | loss/acc = 0.269/0.850
step:212 | loss/acc = 0.240/0.800
step:213 | loss/acc = 0.366/0.800
step:214 | loss/acc = 0.285/0.850
step:215 | loss/acc = 0.113/0.950
step:216 | loss/acc = 0.249/0.900
step:217 | loss/acc = 0.135/0.950
step:218 | loss/acc = 0.216/0.950
step:219 | loss/acc = 0.319/0.950
sentiment task:    219: [    0/    3], senti_loss/senti_acc = 0.23672560/0.8950000 


step:391 | loss/acc = 0.138/0.950
step:392 | loss/acc = 0.345/0.900
step:393 | loss/acc = 0.077/1.000
step:394 | loss/acc = 0.368/0.900
step:395 | loss/acc = 0.079/1.000
step:396 | loss/acc = 0.114/1.000
step:397 | loss/acc = 0.054/1.000
step:398 | loss/acc = 0.186/0.900
step:399 | loss/acc = 0.626/0.750
sentiment task:    399: [    0/    3], senti_loss/senti_acc = 0.21165199/0.9350000 
step:400 | loss/acc = 0.363/0.850
step:401 | loss/acc = 0.285/0.800
step:402 | loss/acc = 0.376/0.850
step:403 | loss/acc = 0.163/0.950
step:404 | loss/acc = 0.176/0.950
step:405 | loss/acc = 0.313/0.950
step:406 | loss/acc = 0.139/0.950
step:407 | loss/acc = 0.226/0.950
step:408 | loss/acc = 0.176/0.950
step:409 | loss/acc = 0.253/0.900
sentiment task:    409: [    0/    3], senti_loss/senti_acc = 0.24682325/0.9100000 
step:410 | loss/acc = 0.174/0.900
step:411 | loss/acc = 0.213/0.900
step:412 | loss/acc = 0.062/1.000
step:413 | loss/acc = 0.163/0.950
step:414 | loss/acc = 0.150/0.900
step:415 | loss/

step:587 | loss/acc = 0.094/0.950
step:588 | loss/acc = 0.357/0.900
step:589 | loss/acc = 0.143/0.950
sentiment task:    589: [    0/    3], senti_loss/senti_acc = 0.18086360/0.9250000 
step:590 | loss/acc = 0.346/0.900
step:591 | loss/acc = 0.036/1.000
step:592 | loss/acc = 0.291/0.850
step:593 | loss/acc = 0.294/0.950
step:594 | loss/acc = 0.518/0.750
step:595 | loss/acc = 0.386/0.850
step:596 | loss/acc = 0.183/0.950
step:597 | loss/acc = 0.203/0.850
step:598 | loss/acc = 0.154/0.950
step:599 | loss/acc = 0.223/0.950
sentiment task:    599: [    0/    3], senti_loss/senti_acc = 0.26323571/0.9000000 
step:600 | loss/acc = 0.238/0.900
step:601 | loss/acc = 0.230/0.900
step:602 | loss/acc = 0.064/1.000
step:603 | loss/acc = 0.226/0.900
step:604 | loss/acc = 0.415/0.900
step:605 | loss/acc = 0.405/0.800
step:606 | loss/acc = 0.043/1.000
step:607 | loss/acc = 0.153/0.950
step:608 | loss/acc = 0.082/0.950
step:609 | loss/acc = 0.239/0.850
sentiment task:    609: [    0/    3], senti_loss/

step:781 | loss/acc = 0.199/0.900
step:782 | loss/acc = 0.053/1.000
step:783 | loss/acc = 0.265/0.900
step:784 | loss/acc = 0.070/1.000
step:785 | loss/acc = 0.228/0.950
step:786 | loss/acc = 0.083/0.950
step:787 | loss/acc = 0.054/1.000
step:788 | loss/acc = 0.059/1.000
step:789 | loss/acc = 0.468/0.800
sentiment task:    789: [    0/    3], senti_loss/senti_acc = 0.17854094/0.9350000 
step:790 | loss/acc = 0.215/0.900


In [17]:
senti_cls_train(senti_train_reader, senti_valid_reader,
                    bert, transformer, task_embedding, senti_cls,
                    10, cuda=True, log_dir="SentiRDM"
                   )

step:0 | loss/acc = 0.730/0.450
step:1 | loss/acc = 1.202/0.550
step:2 | loss/acc = 0.704/0.600
step:3 | loss/acc = 0.633/0.650
step:4 | loss/acc = 0.947/0.550
step:5 | loss/acc = 0.854/0.550
step:6 | loss/acc = 0.788/0.450
step:7 | loss/acc = 0.743/0.450
step:8 | loss/acc = 0.974/0.400
step:9 | loss/acc = 0.732/0.550
sentiment task:      9: [    0/   10], senti_loss/senti_acc = 0.83069446/0.5200000 
step:10 | loss/acc = 0.724/0.450
step:11 | loss/acc = 0.673/0.600
step:12 | loss/acc = 0.570/0.800
step:13 | loss/acc = 0.660/0.600
step:14 | loss/acc = 0.793/0.550
step:15 | loss/acc = 1.136/0.200
step:16 | loss/acc = 0.647/0.650
step:17 | loss/acc = 0.658/0.700
step:18 | loss/acc = 0.686/0.550
step:19 | loss/acc = 0.748/0.450
sentiment task:     19: [    0/   10], senti_loss/senti_acc = 0.72942814/0.5550000 
step:20 | loss/acc = 0.733/0.500
step:21 | loss/acc = 0.680/0.600
step:22 | loss/acc = 0.816/0.350
step:23 | loss/acc = 0.650/0.600
step:24 | loss/acc = 0.678/0.450
step:25 | loss/ac

step:199 | loss/acc = 0.609/0.750
sentiment task:    199: [    0/   10], senti_loss/senti_acc = 0.45052493/0.7800000 
step:200 | loss/acc = 0.575/0.750
step:201 | loss/acc = 0.641/0.750
step:202 | loss/acc = 0.348/0.850
step:203 | loss/acc = 0.574/0.800
step:204 | loss/acc = 0.748/0.550
step:205 | loss/acc = 0.418/0.900
step:206 | loss/acc = 0.482/0.700
step:207 | loss/acc = 0.578/0.750
step:208 | loss/acc = 0.400/0.850
step:209 | loss/acc = 0.522/0.800
sentiment task:    209: [    0/   10], senti_loss/senti_acc = 0.52864830/0.7700000 
step:210 | loss/acc = 0.387/0.800
step:211 | loss/acc = 0.520/0.700
step:212 | loss/acc = 0.489/0.800
step:213 | loss/acc = 0.529/0.700
step:214 | loss/acc = 0.272/0.900
step:215 | loss/acc = 0.399/0.800
step:216 | loss/acc = 0.435/0.750
step:217 | loss/acc = 0.322/0.900
step:218 | loss/acc = 0.484/0.800
step:219 | loss/acc = 0.373/0.850
sentiment task:    219: [    0/   10], senti_loss/senti_acc = 0.42097714/0.8000000 
step:220 | loss/acc = 0.566/0.750


step:392 | loss/acc = 0.362/0.850
step:393 | loss/acc = 0.293/0.900
step:394 | loss/acc = 0.297/0.900
step:395 | loss/acc = 0.493/0.700
step:396 | loss/acc = 0.426/0.750
step:397 | loss/acc = 0.535/0.750
step:398 | loss/acc = 0.330/0.900
step:399 | loss/acc = 0.505/0.700
sentiment task:    399: [    0/   10], senti_loss/senti_acc = 0.42918103/0.8000000 
step:400 | loss/acc = 0.513/0.800
step:401 | loss/acc = 0.513/0.850
step:402 | loss/acc = 0.224/0.950
step:403 | loss/acc = 0.332/0.950
step:404 | loss/acc = 0.425/0.800
step:405 | loss/acc = 0.549/0.600
step:406 | loss/acc = 0.302/0.900
step:407 | loss/acc = 0.367/0.800
step:408 | loss/acc = 0.594/0.750
step:409 | loss/acc = 0.214/0.900
sentiment task:    409: [    0/   10], senti_loss/senti_acc = 0.40320156/0.8300000 
step:410 | loss/acc = 0.316/0.900
step:411 | loss/acc = 0.270/0.900
step:412 | loss/acc = 0.535/0.650
step:413 | loss/acc = 0.403/0.750
step:414 | loss/acc = 0.249/0.900
step:415 | loss/acc = 0.348/0.900
step:416 | loss/

step:585 | loss/acc = 0.196/0.850
step:586 | loss/acc = 0.217/0.900
step:587 | loss/acc = 0.297/0.850
step:588 | loss/acc = 0.239/0.850
step:589 | loss/acc = 0.503/0.900
sentiment task:    589: [    1/   10], senti_loss/senti_acc = 0.12151518/0.8650000 
step:590 | loss/acc = 0.715/0.800
step:591 | loss/acc = 0.196/0.900
step:592 | loss/acc = 0.351/0.850
step:593 | loss/acc = 0.176/0.950
step:594 | loss/acc = 0.188/0.950
step:595 | loss/acc = 0.397/0.850
step:596 | loss/acc = 0.182/0.950
step:597 | loss/acc = 0.347/0.750
step:598 | loss/acc = 0.133/1.000
step:599 | loss/acc = 0.414/0.750
sentiment task:    599: [    1/   10], senti_loss/senti_acc = 0.12394030/0.8750000 
step:600 | loss/acc = 0.570/0.700
step:601 | loss/acc = 0.216/0.900
step:602 | loss/acc = 0.353/0.900
step:603 | loss/acc = 0.226/0.900
step:604 | loss/acc = 0.206/0.950
step:605 | loss/acc = 0.372/0.800
step:606 | loss/acc = 0.170/1.000
step:607 | loss/acc = 0.269/0.900
step:608 | loss/acc = 0.280/0.900
step:609 | loss/

step:780 | loss/acc = 0.106/0.950
step:781 | loss/acc = 0.143/0.950
step:782 | loss/acc = 0.216/0.950
step:783 | loss/acc = 0.713/0.650
step:784 | loss/acc = 0.085/0.950
step:785 | loss/acc = 0.081/1.000
step:786 | loss/acc = 0.273/0.850
step:787 | loss/acc = 0.344/0.800
step:788 | loss/acc = 0.244/0.900
step:789 | loss/acc = 0.332/0.850
sentiment task:    789: [    1/   10], senti_loss/senti_acc = 0.10154458/0.8850000 
step:790 | loss/acc = 0.188/0.950
step:791 | loss/acc = 0.117/0.950
step:792 | loss/acc = 0.354/0.850
step:793 | loss/acc = 0.104/1.000
step:794 | loss/acc = 0.252/0.850
step:795 | loss/acc = 0.091/1.000
step:796 | loss/acc = 0.309/0.800
step:797 | loss/acc = 0.393/0.750
step:798 | loss/acc = 0.310/0.900
step:799 | loss/acc = 0.191/0.950
sentiment task:    799: [    1/   10], senti_loss/senti_acc = 0.09234952/0.9000000 
step:800 | loss/acc = 0.036/1.000
step:801 | loss/acc = 0.163/0.950
step:802 | loss/acc = 0.126/0.950
step:803 | loss/acc = 0.344/0.900
step:804 | loss/

...

step:4200 | loss/acc = 0.002/1.000
step:4201 | loss/acc = 0.017/1.000
step:4202 | loss/acc = 0.008/1.000
step:4203 | loss/acc = 0.002/1.000
step:4204 | loss/acc = 0.002/1.000
step:4205 | loss/acc = 0.004/1.000
step:4206 | loss/acc = 0.005/1.000
step:4207 | loss/acc = 0.021/1.000
step:4208 | loss/acc = 0.009/1.000
step:4209 | loss/acc = 0.011/1.000
sentiment task:   4209: [    8/   10], senti_loss/senti_acc = 0.00318779/1.0000000 
step:4210 | loss/acc = 0.001/1.000
step:4211 | loss/acc = 0.005/1.000
step:4212 | loss/acc = 0.000/1.000
step:4213 | loss/acc = 0.005/1.000
step:4214 | loss/acc = 0.003/1.000
step:4215 | loss/acc = 0.004/1.000
step:4216 | loss/acc = 0.065/0.950
step:4217 | loss/acc = 0.003/1.000
step:4218 | loss/acc = 0.001/1.000
step:4219 | loss/acc = 0.000/1.000
sentiment task:   4219: [    8/   10], senti_loss/senti_acc = 0.00351459/0.9950000 
step:4220 | loss/acc = 0.013/1.000
step:4221 | loss/acc = 0.190/0.900
step:4222 | loss/acc = 0.003/1.000
step:4223 | loss/acc = 0.00

step:4391 | loss/acc = 0.003/1.000
step:4392 | loss/acc = 0.051/0.950
step:4393 | loss/acc = 0.001/1.000
step:4394 | loss/acc = 0.000/1.000
step:4395 | loss/acc = 0.001/1.000
step:4396 | loss/acc = 0.002/1.000
step:4397 | loss/acc = 0.039/0.950
step:4398 | loss/acc = 0.077/0.900
step:4399 | loss/acc = 0.001/1.000
sentiment task:   4399: [    8/   10], senti_loss/senti_acc = 0.01226814/0.9700000 
step:4400 | loss/acc = 0.002/1.000
step:4401 | loss/acc = 0.039/0.950
step:4402 | loss/acc = 0.036/1.000
step:4403 | loss/acc = 0.000/1.000
step:4404 | loss/acc = 0.050/0.950
step:4405 | loss/acc = 0.038/1.000
step:4406 | loss/acc = 0.043/0.950
step:4407 | loss/acc = 0.000/1.000
step:4408 | loss/acc = 0.044/0.950
step:4409 | loss/acc = 0.036/0.950
sentiment task:   4409: [    8/   10], senti_loss/senti_acc = 0.01159094/0.9750000 
step:4410 | loss/acc = 0.000/1.000
step:4411 | loss/acc = 0.001/1.000
step:4412 | loss/acc = 0.001/1.000
step:4413 | loss/acc = 0.000/1.000
step:4414 | loss/acc = 0.00

step:4580 | loss/acc = 0.002/1.000
step:4581 | loss/acc = 0.034/1.000
step:4582 | loss/acc = 0.000/1.000
step:4583 | loss/acc = 0.001/1.000
step:4584 | loss/acc = 0.000/1.000
step:4585 | loss/acc = 0.038/0.950
step:4586 | loss/acc = 0.000/1.000
step:4587 | loss/acc = 0.071/0.950
step:4588 | loss/acc = 0.033/1.000
step:4589 | loss/acc = 0.001/1.000
sentiment task:   4589: [    9/   10], senti_loss/senti_acc = 0.00719676/0.9900000 
step:4590 | loss/acc = 0.115/0.950
step:4591 | loss/acc = 0.000/1.000
step:4592 | loss/acc = 0.000/1.000
step:4593 | loss/acc = 0.052/0.950
step:4594 | loss/acc = 0.003/1.000
step:4595 | loss/acc = 0.020/1.000
step:4596 | loss/acc = 0.002/1.000
step:4597 | loss/acc = 0.002/1.000
step:4598 | loss/acc = 0.001/1.000
step:4599 | loss/acc = 0.106/0.900
sentiment task:   4599: [    9/   10], senti_loss/senti_acc = 0.01203781/0.9800000 
step:4600 | loss/acc = 0.003/1.000
step:4601 | loss/acc = 0.159/0.900
step:4602 | loss/acc = 0.000/1.000
step:4603 | loss/acc = 0.00

step:4770 | loss/acc = 0.024/1.000
step:4771 | loss/acc = 0.002/1.000
step:4772 | loss/acc = 0.002/1.000
step:4773 | loss/acc = 0.013/1.000
step:4774 | loss/acc = 0.004/1.000
step:4775 | loss/acc = 0.001/1.000
step:4776 | loss/acc = 0.001/1.000
step:4777 | loss/acc = 0.013/1.000
step:4778 | loss/acc = 0.013/1.000
step:4779 | loss/acc = 0.018/1.000
sentiment task:   4779: [    9/   10], senti_loss/senti_acc = 0.00362370/1.0000000 
step:4780 | loss/acc = 0.001/1.000
step:4781 | loss/acc = 0.216/0.950
step:4782 | loss/acc = 0.001/1.000
step:4783 | loss/acc = 0.104/0.950
step:4784 | loss/acc = 0.123/0.950
step:4785 | loss/acc = 0.003/1.000
step:4786 | loss/acc = 0.005/1.000
step:4787 | loss/acc = 0.014/1.000
step:4788 | loss/acc = 0.002/1.000
step:4789 | loss/acc = 0.223/0.950
sentiment task:   4789: [    9/   10], senti_loss/senti_acc = 0.02759319/0.9800000 
step:4790 | loss/acc = 0.012/1.000
step:4791 | loss/acc = 0.020/1.000
step:4792 | loss/acc = 0.001/1.000
step:4793 | loss/acc = 0.00

step:4960 | loss/acc = 0.033/1.000
step:4961 | loss/acc = 0.000/1.000
step:4962 | loss/acc = 0.000/1.000
step:4963 | loss/acc = 0.000/1.000
step:4964 | loss/acc = 0.000/1.000
step:4965 | loss/acc = 0.000/1.000
step:4966 | loss/acc = 0.000/1.000
step:4967 | loss/acc = 0.001/1.000
step:4968 | loss/acc = 0.000/1.000
step:4969 | loss/acc = 0.000/1.000
sentiment task:   4969: [    9/   10], senti_loss/senti_acc = 0.00142354/1.0000000 
step:4970 | loss/acc = 0.000/1.000
step:4971 | loss/acc = 0.000/1.000
step:4972 | loss/acc = 0.000/1.000
step:4973 | loss/acc = 0.000/1.000
step:4974 | loss/acc = 0.000/1.000
step:4975 | loss/acc = 0.000/1.000
step:4976 | loss/acc = 0.015/1.000
step:4977 | loss/acc = 0.000/1.000
step:4978 | loss/acc = 0.037/0.950
step:4979 | loss/acc = 0.000/1.000
sentiment task:   4979: [    9/   10], senti_loss/senti_acc = 0.00213466/0.9950000 
step:4980 | loss/acc = 0.036/0.950
step:4981 | loss/acc = 0.041/0.950
step:4982 | loss/acc = 0.000/1.000
step:4983 | loss/acc = 0.03

In [ ]:
senti_cls_train(senti_train_reader, bert, transformer,
                    task_embedding, senti_cls,
                    1, senti_loss_fn,
                    cuda=True
                   )

In [ ]:
joint_model_save_as = '/home/hadoop/ERD/MTLTrain/SentiPretrainModel_epoch.pkl'
torch.save(
    {
        "bert":bert.state_dict(),
        "transformer":transformer.state_dict(),
        "task_embedding":task_embedding.state_dict(),
        "senti_classifier": senti_cls.state_dict(),
#         "subj_classifier": subj_classifier.state_dict(),
#         "emotion_classifier": emotion_classifier.state_dict()
    },
    joint_model_save_as
)

#### 调试emotion classification　任务

In [13]:
file2 = "/home/hadoop/EmoNet-PyTorch/twitter30.json"

In [14]:
sentences, labels, emotion_set = emotionLoader.load_data_from_file(file2)

In [15]:
emoReader = emotionLoader.EmotionReader(sentences[:-400], labels[:-400], 20, tt)
valReader = emotionLoader.EmotionReader(sentences[-400:], labels[-400:], 20, tt)

In [18]:
emoReader.label.shape

(991, 20, 6)

In [19]:
emo_cls = nn.Linear(768, 6).cuda()

In [22]:
emo_cls_train(
        emoReader,
        valReader,
        bert, 
        transformer, 
        task_embedding, 
        emo_cls, 
        5, 
        log_dir="EmoRDM",
        cuda=True
    )

step:0 | loss/acc = 1.666/0.400
step:1 | loss/acc = 1.733/0.300
step:2 | loss/acc = 1.721/0.450
step:3 | loss/acc = 1.170/0.650
step:4 | loss/acc = 1.226/0.600
step:5 | loss/acc = 1.954/0.300
step:6 | loss/acc = 1.907/0.550
step:7 | loss/acc = 1.506/0.500
step:8 | loss/acc = 1.124/0.600
step:9 | loss/acc = 1.705/0.450
emotion task:      9: [    0/    5], senti_loss/senti_acc = 1.57124186/0.4800000 
step:10 | loss/acc = 1.366/0.450
step:11 | loss/acc = 1.644/0.650
step:12 | loss/acc = 1.710/0.350
step:13 | loss/acc = 1.656/0.650
step:14 | loss/acc = 1.844/0.450
step:15 | loss/acc = 1.364/0.400
step:16 | loss/acc = 1.777/0.650
step:17 | loss/acc = 1.604/0.600
step:18 | loss/acc = 1.612/0.500
step:19 | loss/acc = 1.475/0.550
emotion task:     19: [    0/    5], senti_loss/senti_acc = 1.60504158/0.5250000 
step:20 | loss/acc = 1.650/0.450
step:21 | loss/acc = 2.083/0.650
step:22 | loss/acc = 1.664/0.600
step:23 | loss/acc = 1.289/0.550
step:24 | loss/acc = 1.356/0.550
step:25 | loss/acc = 

step:200 | loss/acc = 1.439/0.300
step:201 | loss/acc = 1.451/0.250
step:202 | loss/acc = 1.563/0.300
step:203 | loss/acc = 1.439/0.250
step:204 | loss/acc = 1.583/0.150
step:205 | loss/acc = 1.284/0.350
step:206 | loss/acc = 1.563/0.100
step:207 | loss/acc = 1.564/0.250
step:208 | loss/acc = 1.600/0.350
step:209 | loss/acc = 1.472/0.400
emotion task:    209: [    0/    5], senti_loss/senti_acc = 1.49575347/0.2700000 
step:210 | loss/acc = 1.532/0.200
step:211 | loss/acc = 1.763/0.300
step:212 | loss/acc = 1.811/0.400
step:213 | loss/acc = 1.246/0.400
step:214 | loss/acc = 1.568/0.400
step:215 | loss/acc = 1.520/0.350
step:216 | loss/acc = 1.827/0.150
step:217 | loss/acc = 1.607/0.350
step:218 | loss/acc = 1.618/0.350
step:219 | loss/acc = 1.516/0.350
emotion task:    219: [    0/    5], senti_loss/senti_acc = 1.60070237/0.3250000 
step:220 | loss/acc = 1.574/0.250
step:221 | loss/acc = 1.532/0.350
step:222 | loss/acc = 1.756/0.250
step:223 | loss/acc = 1.865/0.200
step:224 | loss/acc 

step:397 | loss/acc = 1.613/0.200
step:398 | loss/acc = 1.858/0.200
step:399 | loss/acc = 1.758/0.150
emotion task:    399: [    0/    5], senti_loss/senti_acc = 1.64089371/0.3250000 
step:400 | loss/acc = 1.950/0.150
step:401 | loss/acc = 1.581/0.300
step:402 | loss/acc = 1.337/0.400
step:403 | loss/acc = 2.056/0.200
step:404 | loss/acc = 1.874/0.250
step:405 | loss/acc = 2.010/0.200
step:406 | loss/acc = 1.474/0.400
step:407 | loss/acc = 1.515/0.350
step:408 | loss/acc = 1.697/0.300
step:409 | loss/acc = 1.540/0.300
emotion task:    409: [    0/    5], senti_loss/senti_acc = 1.70354962/0.2850000 
step:410 | loss/acc = 1.500/0.400
step:411 | loss/acc = 1.488/0.300
step:412 | loss/acc = 1.941/0.200
step:413 | loss/acc = 1.243/0.450
step:414 | loss/acc = 1.463/0.300
step:415 | loss/acc = 1.560/0.350
step:416 | loss/acc = 1.884/0.250
step:417 | loss/acc = 1.318/0.400
step:418 | loss/acc = 1.418/0.350
step:419 | loss/acc = 1.406/0.500
emotion task:    419: [    0/    5], senti_loss/senti_

step:591 | loss/acc = 1.338/0.600
step:592 | loss/acc = 1.642/0.550
step:593 | loss/acc = 1.907/0.450
step:594 | loss/acc = 1.877/0.600
step:595 | loss/acc = 1.168/0.650
step:596 | loss/acc = 1.586/0.600
step:597 | loss/acc = 1.477/0.550
step:598 | loss/acc = 1.528/0.500
step:599 | loss/acc = 1.279/0.650
emotion task:    599: [    0/    5], senti_loss/senti_acc = 1.50833142/0.5850000 
step:600 | loss/acc = 1.706/0.700
step:601 | loss/acc = 1.557/0.550
step:602 | loss/acc = 1.071/0.750
step:603 | loss/acc = 1.917/0.650
step:604 | loss/acc = 1.321/0.550
step:605 | loss/acc = 1.319/0.700
step:606 | loss/acc = 1.977/0.700
step:607 | loss/acc = 1.682/0.500
step:608 | loss/acc = 1.373/0.500
step:609 | loss/acc = 1.877/0.400
emotion task:    609: [    0/    5], senti_loss/senti_acc = 1.57994279/0.6000000 
step:610 | loss/acc = 1.263/0.600
step:611 | loss/acc = 1.246/0.650
step:612 | loss/acc = 1.664/0.700
step:613 | loss/acc = 1.548/0.500
step:614 | loss/acc = 1.485/0.500
step:615 | loss/acc 

step:787 | loss/acc = 1.413/0.350
step:788 | loss/acc = 1.369/0.250
step:789 | loss/acc = 1.315/0.400
emotion task:    789: [    0/    5], senti_loss/senti_acc = 1.37455446/0.3250000 
step:790 | loss/acc = 1.653/0.150
step:791 | loss/acc = 1.347/0.250
step:792 | loss/acc = 1.621/0.200
step:793 | loss/acc = 1.763/0.250
step:794 | loss/acc = 1.553/0.350
step:795 | loss/acc = 1.759/0.250
step:796 | loss/acc = 1.359/0.350
step:797 | loss/acc = 1.361/0.300
step:798 | loss/acc = 1.314/0.500
step:799 | loss/acc = 1.121/0.450
emotion task:    799: [    0/    5], senti_loss/senti_acc = 1.48508663/0.3050000 
step:800 | loss/acc = 1.545/0.200
step:801 | loss/acc = 1.437/0.150
step:802 | loss/acc = 1.721/0.250
step:803 | loss/acc = 1.270/0.350
step:804 | loss/acc = 1.087/0.350
step:805 | loss/acc = 1.573/0.250
step:806 | loss/acc = 1.622/0.150
step:807 | loss/acc = 1.351/0.350
step:808 | loss/acc = 1.498/0.150
step:809 | loss/acc = 1.332/0.300
emotion task:    809: [    0/    5], senti_loss/senti_

...

step:154 | loss/acc = 0.633/0.750
step:155 | loss/acc = 0.666/0.700
step:156 | loss/acc = 1.466/0.750
step:157 | loss/acc = 0.339/0.850
step:158 | loss/acc = 0.851/0.700
step:159 | loss/acc = 0.741/0.800
emotion task:    159: [    4/    5], senti_loss/senti_acc = 1.07456746/0.7650000 
step:160 | loss/acc = 0.550/0.850
step:161 | loss/acc = 0.467/0.850
step:162 | loss/acc = 0.571/0.750
step:163 | loss/acc = 0.523/0.750
step:164 | loss/acc = 0.840/0.700
step:165 | loss/acc = 0.644/0.700
step:166 | loss/acc = 1.731/0.650
step:167 | loss/acc = 0.676/0.850
step:168 | loss/acc = 0.444/0.700
step:169 | loss/acc = 0.478/0.700
emotion task:    169: [    4/    5], senti_loss/senti_acc = 1.07291895/0.7500000 
step:170 | loss/acc = 0.176/0.950
step:171 | loss/acc = 0.508/0.800
step:172 | loss/acc = 0.462/0.800
step:173 | loss/acc = 0.509/0.700
step:174 | loss/acc = 1.086/0.750
step:175 | loss/acc = 0.826/0.650
step:176 | loss/acc = 1.719/0.700
step:177 | loss/acc = 0.193/0.900
step:178 | loss/acc 

step:350 | loss/acc = 1.093/0.600
step:351 | loss/acc = 0.412/0.850
step:352 | loss/acc = 0.639/0.850
step:353 | loss/acc = 0.391/0.750
step:354 | loss/acc = 0.597/0.700
step:355 | loss/acc = 0.618/0.750
step:356 | loss/acc = 0.844/0.550
step:357 | loss/acc = 0.533/0.750
step:358 | loss/acc = 0.353/0.700
step:359 | loss/acc = 0.624/0.800
emotion task:    359: [    4/    5], senti_loss/senti_acc = 1.03188003/0.7300000 
step:360 | loss/acc = 0.459/0.800
step:361 | loss/acc = 0.748/0.750
step:362 | loss/acc = 0.184/0.900
step:363 | loss/acc = 0.723/0.700
step:364 | loss/acc = 0.957/0.700
step:365 | loss/acc = 0.959/0.600
step:366 | loss/acc = 1.897/0.700
step:367 | loss/acc = 0.389/0.850
step:368 | loss/acc = 0.876/0.600
step:369 | loss/acc = 0.616/0.700
emotion task:    369: [    4/    5], senti_loss/senti_acc = 1.11705572/0.7300000 
step:370 | loss/acc = 0.611/0.750
step:371 | loss/acc = 0.720/0.650
step:372 | loss/acc = 0.870/0.650
step:373 | loss/acc = 0.644/0.800
step:374 | loss/acc 

step:547 | loss/acc = 0.375/0.850
step:548 | loss/acc = 0.727/0.650
step:549 | loss/acc = 0.965/0.600
emotion task:    549: [    4/    5], senti_loss/senti_acc = 1.06830292/0.7850000 
step:550 | loss/acc = 0.265/0.900
step:551 | loss/acc = 0.191/0.900
step:552 | loss/acc = 2.363/0.700
step:553 | loss/acc = 1.017/0.650
step:554 | loss/acc = 0.480/0.850
step:555 | loss/acc = 1.854/0.650
step:556 | loss/acc = 1.624/0.750
step:557 | loss/acc = 0.726/0.800
step:558 | loss/acc = 1.105/0.700
step:559 | loss/acc = 0.611/0.750
emotion task:    559: [    4/    5], senti_loss/senti_acc = 1.23841255/0.7650000 
step:560 | loss/acc = 1.116/0.550
step:561 | loss/acc = 0.737/0.700
step:562 | loss/acc = 0.712/0.650
step:563 | loss/acc = 0.730/0.800
step:564 | loss/acc = 0.488/0.900
step:565 | loss/acc = 0.647/0.850
step:566 | loss/acc = 0.746/0.800
step:567 | loss/acc = 0.560/0.800
step:568 | loss/acc = 0.269/0.850
step:569 | loss/acc = 0.818/0.700
emotion task:    569: [    4/    5], senti_loss/senti_

step:741 | loss/acc = 0.351/0.850
step:742 | loss/acc = 0.533/0.750
step:743 | loss/acc = 1.139/0.700
step:744 | loss/acc = 0.484/0.800
step:745 | loss/acc = 0.422/0.900
step:746 | loss/acc = 0.318/0.800
step:747 | loss/acc = 0.388/0.750
step:748 | loss/acc = 0.446/0.900
step:749 | loss/acc = 0.914/0.800
emotion task:    749: [    4/    5], senti_loss/senti_acc = 1.02318097/0.8100000 
step:750 | loss/acc = 0.389/0.850
step:751 | loss/acc = 0.323/0.900
step:752 | loss/acc = 0.337/0.850
step:753 | loss/acc = 0.528/0.800
step:754 | loss/acc = 0.538/0.900
step:755 | loss/acc = 0.294/0.850
step:756 | loss/acc = 0.400/0.800
step:757 | loss/acc = 0.509/0.800
step:758 | loss/acc = 0.265/0.900
step:759 | loss/acc = 1.001/0.700
emotion task:    759: [    4/    5], senti_loss/senti_acc = 0.95583336/0.8350000 
step:760 | loss/acc = 0.175/0.950
step:761 | loss/acc = 0.383/0.800
step:762 | loss/acc = 0.216/0.900
step:763 | loss/acc = 0.397/0.800
step:764 | loss/acc = 0.313/0.900
step:765 | loss/acc 

step:938 | loss/acc = 0.558/0.750
step:939 | loss/acc = 1.084/0.650
emotion task:    939: [    4/    5], senti_loss/senti_acc = 1.07499390/0.7350000 
step:940 | loss/acc = 0.880/0.800
step:941 | loss/acc = 1.003/0.800
step:942 | loss/acc = 0.794/0.750
step:943 | loss/acc = 0.621/0.850
step:944 | loss/acc = 0.900/0.800
step:945 | loss/acc = 0.806/0.600
step:946 | loss/acc = 0.521/0.650
step:947 | loss/acc = 1.079/0.650
step:948 | loss/acc = 0.290/0.900
step:949 | loss/acc = 0.417/0.750
emotion task:    949: [    4/    5], senti_loss/senti_acc = 1.09229440/0.7550000 
step:950 | loss/acc = 0.576/0.750
step:951 | loss/acc = 1.680/0.800
step:952 | loss/acc = 0.234/0.850
step:953 | loss/acc = 0.944/0.750
step:954 | loss/acc = 0.268/0.850
step:955 | loss/acc = 0.281/0.900
step:956 | loss/acc = 1.829/0.700
step:957 | loss/acc = 0.763/0.800
step:958 | loss/acc = 0.486/0.750
step:959 | loss/acc = 0.177/0.950
emotion task:    959: [    4/    5], senti_loss/senti_acc = 1.08852579/0.8100000 
step:9

### 联合训练部分

#### 导入预训练模型

In [25]:
senti_save_as = '/home/hadoop/ERD/MTLTrain/SentiPretrainModel_epoch.pkl'

checkpoint = torch.load(senti_save_as)

senti_cls.load_state_dict(checkpoint['senti_classifier'])

<All keys matched successfully>

In [26]:
SentiSubj_saved_as =  '/home/hadoop/ERD/MTLTrain/SubjAfterSentiModel_epoch.pkl'
checkpoint = torch.load(SentiSubj_saved_as)
bert.load_state_dict(checkpoint['bert'])
transformer.load_state_dict(checkpoint['transformer'])
task_embedding.load_state_dict(checkpoint['task_embedding'])
subj_cls.load_state_dict(checkpoint['subj_classifier'])

<All keys matched successfully>

In [21]:
joint_save_as = '/home/hadoop/ERD/MTLTrain/jointModel_epoch015.pkl'
checkpoint = torch.load(joint_save_as)
senti_cls.load_state_dict(checkpoint['senti_classifier'])
bert.load_state_dict(checkpoint['bert'])
transformer.load_state_dict(checkpoint['transformer'])
task_embedding.load_state_dict(checkpoint['task_embedding'])
subj_cls.load_state_dict(checkpoint['subj_classifier'])

<All keys matched successfully>

#### 联合训练

In [28]:
SentiSubjLearning(senti_train_reader, subj_train_reader, 
                  bert, transformer, task_embedding, 
                  senti_cls, subj_cls,
                  cuda=True
                 )

     0    425|MTL_Loss:0.29332089, senti_loss/senti_acc = 0.05127532/0.0750000 | subj_loss/subj_acc = 0.00738885/0.1000000 
     1    425|MTL_Loss:0.19184102, senti_loss/senti_acc = 0.08910429/0.1500000 | subj_loss/subj_acc = 0.00792809/0.2000000 
     2    425|MTL_Loss:0.41711080, senti_loss/senti_acc = 0.15514933/0.2150000 | subj_loss/subj_acc = 0.02530521/0.2850000 
     3    425|MTL_Loss:0.18992978, senti_loss/senti_acc = 0.18254753/0.3150000 | subj_loss/subj_acc = 0.03589297/0.3800000 
     4    425|MTL_Loss:0.26974079, senti_loss/senti_acc = 0.22672767/0.4000000 | subj_loss/subj_acc = 0.04566098/0.4800000 
     5    425|MTL_Loss:0.22977003, senti_loss/senti_acc = 0.26549595/0.4800000 | subj_loss/subj_acc = 0.05284671/0.5750000 
     6    425|MTL_Loss:0.36548451, senti_loss/senti_acc = 0.32818922/0.5450000 | subj_loss/subj_acc = 0.06325034/0.6700000 
     7    425|MTL_Loss:0.31482914, senti_loss/senti_acc = 0.38309467/0.6200000 | subj_loss/subj_acc = 0.07131071/0.7650000 
     8  

KeyboardInterrupt: 

In [ ]:
JointLearning(senti_train_reader, subj_train_reader, emoReader, 
                  bert, transformer, task_embedding, 
                  senti_cls, subj_cls, emo_cls,
                  cuda=True
                 )

step:0 | loss/acc = 0.702/0.500
step:1 | loss/acc = 0.696/0.450
step:2 | loss/acc = 0.697/0.500
step:3 | loss/acc = 0.690/0.500
step:4 | loss/acc = 0.687/0.450
step:5 | loss/acc = 0.668/0.600
step:6 | loss/acc = 0.692/0.500
step:7 | loss/acc = 0.743/0.350
step:8 | loss/acc = 0.651/0.650
step:9 | loss/acc = 0.648/0.750
sentiment task:      9: [    0/    2], senti_loss/senti_acc = 0.68745612/0.5250000 
step:10 | loss/acc = 0.708/0.450
step:11 | loss/acc = 0.664/0.700
step:12 | loss/acc = 0.658/0.750
step:13 | loss/acc = 0.681/0.400
step:14 | loss/acc = 0.672/0.600
step:15 | loss/acc = 0.706/0.450
step:16 | loss/acc = 0.737/0.450
step:17 | loss/acc = 0.781/0.400
step:18 | loss/acc = 0.742/0.450
step:19 | loss/acc = 0.703/0.500
sentiment task:     19: [    0/    2], senti_loss/senti_acc = 0.70535051/0.5150000 
step:20 | loss/acc = 0.784/0.400
step:21 | loss/acc = 0.757/0.450
step:22 | loss/acc = 0.689/0.450
step:23 | loss/acc = 0.720/0.400
step:24 | loss/acc = 0.664/0.650
step:25 | loss/ac

step:198 | loss/acc = 0.631/0.800
step:199 | loss/acc = 0.658/0.550
sentiment task:    199: [    0/    2], senti_loss/senti_acc = 0.63688825/0.6850000 
step:200 | loss/acc = 0.669/0.600
step:201 | loss/acc = 0.604/0.750
step:202 | loss/acc = 0.628/0.750
step:203 | loss/acc = 0.574/0.700
step:204 | loss/acc = 0.590/0.700
step:205 | loss/acc = 0.649/0.600
step:206 | loss/acc = 0.694/0.450
step:207 | loss/acc = 0.741/0.350
step:208 | loss/acc = 0.583/0.750
step:209 | loss/acc = 0.693/0.500
sentiment task:    209: [    0/    2], senti_loss/senti_acc = 0.64253100/0.6150000 
step:210 | loss/acc = 0.694/0.500
step:211 | loss/acc = 0.580/0.750
step:212 | loss/acc = 0.652/0.650
step:213 | loss/acc = 0.713/0.500
step:214 | loss/acc = 0.623/0.700
step:215 | loss/acc = 0.646/0.650
step:216 | loss/acc = 0.656/0.550
step:217 | loss/acc = 0.611/0.800
step:218 | loss/acc = 0.587/0.750
step:219 | loss/acc = 0.642/0.750
sentiment task:    219: [    0/    2], senti_loss/senti_acc = 0.64036837/0.6600000 


step:390 | loss/acc = 0.598/0.750
step:391 | loss/acc = 0.631/0.600
step:392 | loss/acc = 0.542/0.800
step:393 | loss/acc = 0.530/0.850
step:394 | loss/acc = 0.581/0.650
step:395 | loss/acc = 0.700/0.550
step:396 | loss/acc = 0.587/0.650
step:397 | loss/acc = 0.513/0.700
step:398 | loss/acc = 0.500/0.950
step:399 | loss/acc = 0.460/0.850
sentiment task:    399: [    0/    2], senti_loss/senti_acc = 0.56433112/0.7350000 
step:400 | loss/acc = 0.535/0.800
step:401 | loss/acc = 0.685/0.650
step:402 | loss/acc = 0.519/0.850
step:403 | loss/acc = 0.617/0.600
step:404 | loss/acc = 0.568/0.750
step:405 | loss/acc = 0.519/0.800
step:406 | loss/acc = 0.526/0.800
step:407 | loss/acc = 0.635/0.700
step:408 | loss/acc = 0.602/0.750
step:409 | loss/acc = 0.560/0.750
sentiment task:    409: [    0/    2], senti_loss/senti_acc = 0.57669491/0.7450000 
step:410 | loss/acc = 0.550/0.800
step:411 | loss/acc = 0.567/0.750
step:412 | loss/acc = 0.425/0.950
step:413 | loss/acc = 0.654/0.550
step:414 | loss/

step:87 | loss/acc = 0.577/0.800
step:88 | loss/acc = 0.486/0.800
step:89 | loss/acc = 0.461/0.750
sentiment task:     89: [    1/    2], senti_loss/senti_acc = 0.50369355/0.7850000 
step:90 | loss/acc = 0.435/0.900
step:91 | loss/acc = 0.541/0.750
step:92 | loss/acc = 0.382/0.950
step:93 | loss/acc = 0.541/0.800
step:94 | loss/acc = 0.497/0.850
step:95 | loss/acc = 0.339/0.900
step:96 | loss/acc = 0.543/0.850
step:97 | loss/acc = 0.487/0.750
step:98 | loss/acc = 0.602/0.650
step:99 | loss/acc = 0.579/0.700
sentiment task:     99: [    1/    2], senti_loss/senti_acc = 0.49462225/0.8100000 
step:100 | loss/acc = 0.532/0.650
step:101 | loss/acc = 0.578/0.700
step:102 | loss/acc = 0.443/0.800
step:103 | loss/acc = 0.563/0.750
step:104 | loss/acc = 0.427/0.800
step:105 | loss/acc = 0.509/0.750
step:106 | loss/acc = 0.638/0.650
step:107 | loss/acc = 0.469/0.750
step:108 | loss/acc = 0.514/0.700
step:109 | loss/acc = 0.451/0.900
sentiment task:    109: [    1/    2], senti_loss/senti_acc = 0

step:280 | loss/acc = 0.385/0.850
step:281 | loss/acc = 0.461/0.750
step:282 | loss/acc = 0.591/0.600
step:283 | loss/acc = 0.672/0.650
step:284 | loss/acc = 0.457/0.850
step:285 | loss/acc = 0.438/0.700
step:286 | loss/acc = 0.277/0.950
step:287 | loss/acc = 0.541/0.750
step:288 | loss/acc = 0.629/0.650
step:289 | loss/acc = 0.381/0.900
sentiment task:    289: [    1/    2], senti_loss/senti_acc = 0.48315102/0.7650000 
step:290 | loss/acc = 0.478/0.800
step:291 | loss/acc = 0.533/0.800
step:292 | loss/acc = 0.385/0.850
step:293 | loss/acc = 0.513/0.750
step:294 | loss/acc = 0.352/0.850
step:295 | loss/acc = 0.633/0.650
step:296 | loss/acc = 0.531/0.800
step:297 | loss/acc = 0.239/0.900
step:298 | loss/acc = 0.531/0.700
step:299 | loss/acc = 0.526/0.750
sentiment task:    299: [    1/    2], senti_loss/senti_acc = 0.47205535/0.7850000 
step:300 | loss/acc = 0.450/0.750
step:301 | loss/acc = 0.520/0.700
step:302 | loss/acc = 0.368/0.900
step:303 | loss/acc = 0.356/0.900
step:304 | loss/

step:475 | loss/acc = 0.368/0.850
step:476 | loss/acc = 0.499/0.750
step:477 | loss/acc = 0.442/0.750
step:478 | loss/acc = 0.394/0.850
step:479 | loss/acc = 0.356/0.900
sentiment task:    479: [    1/    2], senti_loss/senti_acc = 0.39819183/0.8300000 
step:480 | loss/acc = 0.408/0.800
step:481 | loss/acc = 0.575/0.650
step:482 | loss/acc = 0.510/0.750
step:483 | loss/acc = 0.613/0.750
step:484 | loss/acc = 0.486/0.700
step:485 | loss/acc = 0.380/0.700
step:486 | loss/acc = 0.399/0.800
step:487 | loss/acc = 0.575/0.700
step:488 | loss/acc = 0.563/0.700
step:489 | loss/acc = 0.494/0.800
sentiment task:    489: [    1/    2], senti_loss/senti_acc = 0.50039234/0.7350000 
step:490 | loss/acc = 0.359/0.850
step:491 | loss/acc = 0.769/0.650
step:492 | loss/acc = 0.460/0.750
step:493 | loss/acc = 0.503/0.650
step:494 | loss/acc = 0.401/0.850
step:495 | loss/acc = 0.386/0.750
step:496 | loss/acc = 0.687/0.700
step:497 | loss/acc = 0.674/0.700
step:498 | loss/acc = 0.626/0.650
step:499 | loss/

step:170 | loss/acc = 0.472/0.850
step:171 | loss/acc = 0.090/1.000
step:172 | loss/acc = 0.135/0.950
step:173 | loss/acc = 0.114/0.950
step:174 | loss/acc = 0.190/0.900
step:175 | loss/acc = 0.065/1.000
step:176 | loss/acc = 0.274/0.850
step:177 | loss/acc = 0.114/0.950
step:178 | loss/acc = 0.060/0.950
step:179 | loss/acc = 0.187/0.950
subjective task:    179: [    0/    2], senti_loss/senti_acc = 0.17018927/0.9350000 
step:180 | loss/acc = 0.445/0.900
step:181 | loss/acc = 0.049/1.000
step:182 | loss/acc = 0.084/0.950
step:183 | loss/acc = 0.491/0.800
step:184 | loss/acc = 0.171/0.900
step:185 | loss/acc = 0.259/0.900
step:186 | loss/acc = 0.242/0.950
step:187 | loss/acc = 0.072/0.950
step:188 | loss/acc = 0.072/0.950
step:189 | loss/acc = 0.163/0.950
subjective task:    189: [    0/    2], senti_loss/senti_acc = 0.20488959/0.9250000 
step:190 | loss/acc = 0.200/0.800
step:191 | loss/acc = 0.118/0.950
step:192 | loss/acc = 0.034/1.000
step:193 | loss/acc = 0.463/0.900
step:194 | los

step:364 | loss/acc = 0.251/0.900
step:365 | loss/acc = 0.177/0.900
step:366 | loss/acc = 0.098/0.950
step:367 | loss/acc = 0.115/1.000
step:368 | loss/acc = 0.178/0.900
step:369 | loss/acc = 0.160/0.950
subjective task:    369: [    0/    2], senti_loss/senti_acc = 0.20711721/0.9200000 
step:370 | loss/acc = 0.040/1.000
step:371 | loss/acc = 0.091/0.950
step:372 | loss/acc = 0.072/1.000
step:373 | loss/acc = 0.496/0.850
step:374 | loss/acc = 0.025/1.000
step:375 | loss/acc = 0.072/1.000
step:376 | loss/acc = 0.280/0.900
step:377 | loss/acc = 0.064/1.000
step:378 | loss/acc = 0.122/0.950
step:379 | loss/acc = 0.092/1.000
subjective task:    379: [    0/    2], senti_loss/senti_acc = 0.13545308/0.9650000 
step:380 | loss/acc = 0.021/1.000
step:381 | loss/acc = 0.105/0.950
step:382 | loss/acc = 0.056/1.000
step:383 | loss/acc = 0.041/1.000
step:384 | loss/acc = 0.099/0.900
step:385 | loss/acc = 0.234/0.900
step:386 | loss/acc = 0.012/1.000
step:387 | loss/acc = 0.097/0.950
step:388 | los

In [22]:
rdm_model = RDM_Model(768, 300, 256, 0.2).cuda()
cm_model = CM_Model(300, 256, 2).cuda()
rdm_classifier = nn.Linear(256, 2).cuda()

/home/hadoop/.conda/envs/py37_torch/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
MTLTrainRDMModel(rdm_model, bert, rdm_classifier,
                     transformer, task_embedding, senti_cls, subj_cls, 
                     senti_train_reader, subj_train_reader, 
                    tt, 20000, new_data_len=[], logger=None, cuda=True, 
                        log_dir="RDMBertTrain")

     0  20000|MTL_Loss:0.69997206, rdm_loss/rdm_acc = 0.72082072/0.0000000 | senti_loss/senti_acc = 0.56915508/0.7000000 | subj_loss/subj_acc = 0.66399979/0.7000000 
     1  20000|MTL_Loss:0.76249111, rdm_loss/rdm_acc = 0.77836663/0.5000000 | senti_loss/senti_acc = 0.64321386/0.7000000 | subj_loss/subj_acc = 0.75476415/0.6000000 
     2  20000|MTL_Loss:0.68839521, rdm_loss/rdm_acc = 0.65440810/1.0000000 | senti_loss/senti_acc = 0.78327168/0.4000000 | subj_loss/subj_acc = 0.86541580/0.4000000 
     3  20000|MTL_Loss:0.63432469, rdm_loss/rdm_acc = 0.60354131/1.0000000 | senti_loss/senti_acc = 0.78165638/0.5000000 | subj_loss/subj_acc = 0.73325985/0.6000000 
     4  20000|MTL_Loss:0.77588555, rdm_loss/rdm_acc = 0.80867416/0.5000000 | senti_loss/senti_acc = 0.58763288/0.7000000 | subj_loss/subj_acc = 0.70182917/0.6000000 
     5  20000|MTL_Loss:0.77552871, rdm_loss/rdm_acc = 0.74261606/0.0000000 | senti_loss/senti_acc = 0.76038471/0.5000000 | subj_loss/subj_acc = 1.05397377/0.3000000 
    

## 调试CM模型

In [9]:
import sys

sys.path.append(".")

from logger import MyLogger
import time
import SubjObjLoader
import json
from torch import nn
import torch
from pytorch_transformers import *
import importlib
from collections import deque
# import dataloader
from BertRDMLoader import *
import json
from torch import nn
import torch
from pytorch_transformers import *
import importlib
from tensorboardX import SummaryWriter
import torch.nn.utils.rnn as rnn_utils
import tsentiLoader
import pickle
import tqdm

import os

In [15]:

with open("config.json", "r") as cr:
    dic = json.load(cr)

class adict(dict):
    ''' Attribute dictionary - a convenience data structure, similar to SimpleNamespace in python 3.3
        One can use attributes to read/write dictionary content.
    '''
    def __init__(self, *av, **kav):
        dict.__init__(self, *av, **kav)
        self.__dict__ = self

FLAGS = adict(dic)

load_data_fast()


# #### 创建模型
tokenizer = BertTokenizer.from_pretrained("./bertModel/")
bert = BertModel.from_pretrained("./bertModel/").cuda()
rdm_model = RDM_Model(768, 300, 256, 0.2).cuda()
cm_model = CM_Model_V1(256, 2).cuda()
rdm_classifier = nn.Linear(256, 2).cuda()
cm_log_dir="CMBertTrain"

max_sent: 187 ,  max_seq_len: 101
5802 data loaded


/home/hadoop/.conda/envs/torch_B/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [16]:
load_test_data_fast()

max_sent: 187 ,  max_seq_len: 101
5802 data loaded


In [17]:
log_dir = os.path.join("./runing_models/BertRDM/")
log_dir

'./runing_models/BertRDM/'

In [18]:
pretrained_file = "%s/BertRDM_best.pkl"%log_dir
if os.path.exists(pretrained_file):
    checkpoint = torch.load(pretrained_file)
    bert.load_state_dict(checkpoint['bert'])
    rdm_model.load_state_dict(checkpoint["rmdModel"])
    rdm_classifier.load_state_dict(checkpoint["rdm_classifier"])
else:
    TrainRDMModel(rdm_model, bert, rdm_classifier, 
                    tokenizer, 2000, stage=0, new_data_len=[], logger=None, 
                        log_dir=log_dir, cuda=True)


In [12]:

def TrainCMModel(bert, rdm_model, rdm_classifier, cm_model, tokenizer, stage, t_rw, t_steps, log_dir, logger, FLAGS, cuda=False):
    batch_size = 20
    t_acc = 0.9
    ids = np.array(range(batch_size), dtype=np.int32)
    seq_states = np.zeros([batch_size], dtype=np.int32)
    isStop = np.zeros([batch_size], dtype=np.int32)
    max_id = batch_size
    df_init_states = torch.zeros([1, batch_size, rdm_model.hidden_dim], dtype=torch.float32).cuda()
    writer = SummaryWriter(log_dir, filename_suffix="_ERD_CM_stage_%3d"%stage)
    state = df_init_states
    D = deque()
    ssq = []
    print("in RL the begining")
    rl_optim = torch.optim.Adam([{'params':cm_model.parameters(), 'lr':1e-5}])
    # get_new_len(sess, mm)
    data_ID = get_data_ID()

    if len(data_ID) % batch_size == 0: # the total number of events
            flags = int(len(data_ID) / FLAGS.batch_size)
    else:
        flags = int(len(data_ID) / FLAGS.batch_size) + 1

    rdm_hiddens_seq = []
    for i in range(flags):
        with torch.no_grad():
            x, x_len, y = get_df_batch(i, batch_size, tokenizer=tokenizer)
            sent_tensors, attn_mask, seq_len = rdm_data2bert_tensors(x, cuda)
            bert_outs = bert(sent_tensors, attention_mask=attn_mask)
            pooled_sents = [bert_outs[1][sum(seq_len[:idx]):sum(seq_len[:idx])+seq_len[idx]] for idx, s_len in enumerate(seq_len)]
            x_emb = rnn_utils.pad_sequence(pooled_sents, batch_first=True).unsqueeze(-2)
            batchsize, max_seq_len, max_sent_len, emb_dim = x_emb.shape
            rdm_hiddens = rdm_model(x_emb)
            batchsize, _, _ = rdm_hiddens.shape
            tmp_hiddens = [ rdm_hiddens[i][:x_len[i]] for i in range(batchsize)] 

            rdm_hiddens_seq.extend(tmp_hiddens)
            print("batch %d"%i)
            if len(ssq) > 0:
                ssq.extend([rdm_classifier(h) for h in tmp_hiddens])
            else:
                ssq = [rdm_classifier(h) for h in tmp_hiddens]
            torch.cuda.empty_cache()
    del rdm_hiddens, tmp_hiddens

    print(get_curtime() + " Now Start RL training ...")

    counter = 0
    sum_rw = 0.0 # sum of rewards
    data_len = get_data_len()

    while True:
    #         if counter > FLAGS.OBSERVE:
        if counter > 1000:
            sum_rw += rw.mean()
            if counter % 200 == 0:
                sum_rw = float(sum_rw) / 200
                print( get_curtime() + " Step: " + str(counter) 
                       + " REWARD IS " + str(sum_rw) 
                     )
                # if sum_rw > t_rw:
                #     print("Retch The Target Reward")
                #     break
                if counter > t_steps:
                    print("Retch The Target Steps")
                    break
                sum_rw = 0.0
            rdm_state, s_ids, s_seq_states = get_RL_Train_batch_V1(D, FLAGS, batch_size, cuda)

            with torch.no_grad():
                s_stopScore, s_isStop = cm_model(rdm_state)                
                rw, q_val = get_reward_0(s_isStop, s_stopScore, ssq, s_ids, s_seq_states)

            s_isStop = s_stopScore.argsort()
            for j in range(batch_size):
                if random.random() < FLAGS.random_rate:
                    s_isStop[j][int(torch.rand(2).argmax())] = 1 # 设置了一个随机的干扰。
                if seq_states[j] == data_len[s_ids[j]]:
                    s_isStop[j] = 1

            t_stopScore, t_isStop = cm_model(rdm_state)
            out_action = (t_stopScore*s_isStop.float()).sum(axis=1)
            rl_cost = torch.mean((q_val.cuda() - out_action)*(q_val.cuda() - out_action))
            rl_cost.backward()
            torch.cuda.empty_cache()
            rl_optim.step()
            print("RL Cost:", rl_cost)
            writer.add_scalar('RL Cost', rl_cost, counter - FLAGS.OBSERVE)

        input_x, input_y, ids, seq_states, max_id = get_rl_batch(ids, seq_states, isStop, max_id, 0, FLAGS, tokenizer=tokenizer)
        if counter > FLAGS.OBSERVE:
            print("step:", counter - FLAGS.OBSERVE, ", reward:", rw.mean())
            print("step:", counter - FLAGS.OBSERVE, ", reward:", q_val.mean())
            writer.add_scalar('reward', rw.mean(), counter - FLAGS.OBSERVE)
        for j in range(batch_size):
            D.append((rdm_hiddens_seq[ids[j]][seq_states[j]-1], ids[j], seq_states[j]))
            if len(D) > FLAGS.max_memory:
                D.popleft()
        counter += 1